In [ ]:
## This file implements neural networks before and after lasso selection for p0040presabsSTCC_qual with four replicates.
## We compute the mean and standarad deviation of training and test accuracies.
## We also compute the mean and standard deviation of AUC ROC values for each model.

In [1]:
from numpy.random import seed
import numpy as np
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [2]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/p0040presabsSTCC_qual.csv')
df.shape

(253, 353)

In [3]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [4]:
df['pheno']

0      2
1      0
2      2
3      2
4      2
      ..
248    2
249    1
250    1
251    2
252    2
Name: pheno, Length: 253, dtype: int64

In [5]:
df.head()

,id,TTTTTATATTGCGAAAAATAATTGGCGAACGAGGTAACTGGATACCTCATCCGCCAATTAAAATTTGTTAATTTAATAATTAAATATAAAGACGATTTAT,TTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGT,TTTTACAAAAGAGGAAGCAGATAAACTTTATCAACCTATCGGTTCTTCGCAGCCGTCACTGAATATTTGGACAGGCAGTGAAACAGAATATAATTATTTG,TTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTTAGTAGAGTG,TTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTC,TTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTT,TTTGGACAGGCAGTGAAAC,TTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATC,TTTCGCTGTTGCAACATCTTTAACAGGTGTTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACT,...,group_8800,group_10077,group_1598,group_8093,group_8182,group_8892,group_9858,ST,CC,pheno
0,107,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,5,5,2
1,109,1,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,8,8,0
2,115,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,5,5,2
3,120335,0,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,2
4,120337,0,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,2


In [6]:
df['pheno'].value_counts()

2    95
1    94
0    64
Name: pheno, dtype: int64

In [7]:
df_clean = df.drop(columns=['id'])

In [8]:
df_clean.shape

(253, 352)

In [9]:
df_clean.head()

,TTTTTATATTGCGAAAAATAATTGGCGAACGAGGTAACTGGATACCTCATCCGCCAATTAAAATTTGTTAATTTAATAATTAAATATAAAGACGATTTAT,TTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGT,TTTTACAAAAGAGGAAGCAGATAAACTTTATCAACCTATCGGTTCTTCGCAGCCGTCACTGAATATTTGGACAGGCAGTGAAACAGAATATAATTATTTG,TTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTTAGTAGAGTG,TTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTC,TTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTT,TTTGGACAGGCAGTGAAAC,TTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATC,TTTCGCTGTTGCAACATCTTTAACAGGTGTTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACT,TTTCAGCGACTT,...,group_8800,group_10077,group_1598,group_8093,group_8182,group_8892,group_9858,ST,CC,pheno
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,5,5,2
1,1,0,1,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,8,8,0
2,0,0,1,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,5,5,2
3,0,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,2
4,0,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,2


In [10]:
X = df.loc[:, df.columns != 'pheno']
y = df['pheno']
print(X.shape, y.shape)

(253, 352) (253,)


In [11]:
############# Fully-Connected Neural Network ################

In [12]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1

Using TensorFlow backend.


In [13]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y)

In [14]:
dat = pd.DataFrame(X_test.iloc[:,0])
dat['test'] = y_test

In [15]:
dat

,id,test
221,NRS266,2
177,NRS209,1
223,NRS272,2
49,CFBREBSa114,2
53,CFBREBSa119,1
...,...,...
162,NRS187,1
237,SR1287,0
84,CFBRSa50,0
168,NRS196,2


In [16]:
X_train = X_train.drop(['id'], axis=1)
X_test = X_test.drop(['id'], axis=1)

In [17]:
#### neural network on over-sampling data
model1 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(3, activation='softmax'),
])

In [18]:
model1.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [19]:
model1.fit(X_train, y_train,
          batch_size=32, epochs=100,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 615us/step - loss: 23.5059 - accuracy: 0.2147 - val_loss: 12.7146 - val_accuracy: 0.1974
Epoch 2/100
177/177 [==============================] - 0s 96us/step - loss: 16.4104 - accuracy: 0.2881 - val_loss: 8.2075 - val_accuracy: 0.3158
Epoch 3/100
177/177 [==============================] - 0s 141us/step - loss: 11.3311 - accuracy: 0.3164 - val_loss: 5.3159 - val_accuracy: 0.3553
Epoch 4/100
177/177 [==============================] - 0s 152us/step - loss: 8.7051 - accuracy: 0.3446 - val_loss: 4.3300 - val_accuracy: 0.4211
Epoch 5/100
177/177 [==============================] - 0s 148us/step - loss: 7.2854 - accuracy: 0.4181 - val_loss: 3.0499 - val_accuracy: 0.4211
Epoch 6/100
177/177 [==============================] - 0s 143us/step - loss: 4.9591 - accuracy: 0.4068 - val_loss: 2.3373 - val_accuracy: 0.5000
Epoch 7/100
177/177 [==============================] - 0s 163us/step - loss: 3.777

Epoch 57/100
177/177 [==============================] - 0s 139us/step - loss: 0.5343 - accuracy: 0.8305 - val_loss: 0.8937 - val_accuracy: 0.5526
Epoch 58/100
177/177 [==============================] - 0s 130us/step - loss: 0.5373 - accuracy: 0.8475 - val_loss: 0.8582 - val_accuracy: 0.5921
Epoch 59/100
177/177 [==============================] - 0s 147us/step - loss: 0.5718 - accuracy: 0.8249 - val_loss: 1.1325 - val_accuracy: 0.5921
Epoch 60/100
177/177 [==============================] - 0s 97us/step - loss: 0.7553 - accuracy: 0.8249 - val_loss: 1.0689 - val_accuracy: 0.5789
Epoch 61/100
177/177 [==============================] - 0s 94us/step - loss: 0.5286 - accuracy: 0.8588 - val_loss: 0.8555 - val_accuracy: 0.5789
Epoch 62/100
177/177 [==============================] - 0s 99us/step - loss: 0.4633 - accuracy: 0.8644 - val_loss: 0.8326 - val_accuracy: 0.6184
Epoch 63/100
177/177 [==============================] - 0s 110us/step - loss: 0.4666 - accuracy: 0.8644 - val_loss: 0.8478 - va

In [32]:
acc_test1 = model1.evaluate(X_test, y_test)[1]
print('test accuracy: %.2f%%' % (acc_test1*100))

76/76 [==============================] - 0s 79us/step
test accuracy: 61.84%


In [20]:
pred = model1.predict_classes(X_test)
pred

array([1, 1, 1, 2, 2, 0, 2, 1, 2, 2, 1, 2, 0, 1, 1, 2, 0, 1, 1, 0, 2, 2,
       1, 1, 2, 1, 2, 2, 2, 1, 1, 0, 2, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 0,
       2, 2, 1, 1, 0, 2, 0, 1, 0, 2, 1, 0, 2, 2, 0, 2, 0, 1, 0, 1, 2, 0,
       2, 0, 1, 2, 1, 1, 1, 0, 2, 0])

In [21]:
dat['pred'] = pred
dat

,id,test,pred
221,NRS266,2,1
177,NRS209,1,1
223,NRS272,2,1
49,CFBREBSa114,2,2
53,CFBREBSa119,1,2
...,...,...,...
162,NRS187,1,1
237,SR1287,0,1
84,CFBRSa50,0,0
168,NRS196,2,2


In [22]:
proba1 = model1.predict_proba(X_test)
dat_proba1 = pd.DataFrame(proba1)

In [23]:
dat_proba1

,0,1,2
0,0.000033,0.621460,0.378507
1,0.023729,0.973869,0.002402
2,0.020596,0.897287,0.082117
3,0.022055,0.257970,0.719975
4,0.003793,0.053047,0.943160
...,...,...,...
71,0.216843,0.568123,0.215034
72,0.418802,0.553160,0.028038
73,0.948100,0.035031,0.016869
74,0.000964,0.039095,0.959940


In [24]:
dat_proba1.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba1.csv", index = False,
         header=None)

In [25]:
dat.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/1p40pST.csv", index = False,
         header=None)

In [36]:
hist1 = model1.fit(X_train, y_train,
          batch_size=32, epochs=100,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 151us/step - loss: 0.2791 - accuracy: 0.9322 - val_loss: 0.9579 - val_accuracy: 0.6053
Epoch 2/100
177/177 [==============================] - 0s 139us/step - loss: 0.2981 - accuracy: 0.9266 - val_loss: 0.9306 - val_accuracy: 0.6316
Epoch 3/100
177/177 [==============================] - 0s 177us/step - loss: 0.3080 - accuracy: 0.9322 - val_loss: 0.9278 - val_accuracy: 0.6184
Epoch 4/100
177/177 [==============================] - 0s 143us/step - loss: 0.3252 - accuracy: 0.9322 - val_loss: 0.9654 - val_accuracy: 0.6053
Epoch 5/100
177/177 [==============================] - 0s 133us/step - loss: 0.2661 - accuracy: 0.9266 - val_loss: 1.0083 - val_accuracy: 0.5921
Epoch 6/100
177/177 [==============================] - 0s 153us/step - loss: 0.2927 - accuracy: 0.9322 - val_loss: 0.9266 - val_accuracy: 0.6316
Epoch 7/100
177/177 [==============================] - 0s 124us/step - loss: 0.2967 -

Epoch 57/100
177/177 [==============================] - 0s 179us/step - loss: 0.2945 - accuracy: 0.9492 - val_loss: 1.1570 - val_accuracy: 0.6184
Epoch 58/100
177/177 [==============================] - 0s 269us/step - loss: 0.1739 - accuracy: 0.9492 - val_loss: 1.0361 - val_accuracy: 0.6184
Epoch 59/100
177/177 [==============================] - 0s 182us/step - loss: 0.2724 - accuracy: 0.9548 - val_loss: 1.0872 - val_accuracy: 0.5921
Epoch 60/100
177/177 [==============================] - 0s 169us/step - loss: 0.1871 - accuracy: 0.9492 - val_loss: 1.1286 - val_accuracy: 0.6184
Epoch 61/100
177/177 [==============================] - 0s 176us/step - loss: 0.2718 - accuracy: 0.9435 - val_loss: 1.0751 - val_accuracy: 0.6053
Epoch 62/100
177/177 [==============================] - 0s 198us/step - loss: 0.2000 - accuracy: 0.9492 - val_loss: 1.4353 - val_accuracy: 0.5921
Epoch 63/100
177/177 [==============================] - 0s 504us/step - loss: 0.4476 - accuracy: 0.9266 - val_loss: 1.2447 -

In [37]:
print('train accuracy: %.2f%%' % (np.mean(hist1.history['accuracy'])*100))

train accuracy: 94.62%


In [17]:
df_proba = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_2.xlsx",
                        sheet_name=0,
                        index_col=None)

In [18]:
df_proba

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,NRS027,1,0,0.759813,0.001304,0.238883
1,p0017kpresabs_qual,NRS246,1,0,0.947819,0.009591,0.042591
2,p0017kpresabs_qual,NRS218,2,1,0.008693,0.989390,0.001916
3,p0017kpresabs_qual,CFBRSa70,2,0,0.813774,0.001482,0.184744
4,p0017kpresabs_qual,NRS177,1,1,0.000916,0.998926,0.000157
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,NRS187,1,1,0.216843,0.568123,0.215034
604,p0040presabsSTCC_qual,SR1287,0,1,0.418802,0.553160,0.028038
605,p0040presabsSTCC_qual,CFBRSa50,0,0,0.948100,0.035031,0.016869
606,p0040presabsSTCC_qual,NRS196,2,2,0.000964,0.039095,0.959940


In [19]:
y_prob = df_proba[df_proba['phage']=='p0040presabsSTCC_qual'].iloc[:,-3:]
y_prob = y_prob.to_numpy()
y_prob

array([[3.26926200e-05, 6.21460500e-01, 3.78506870e-01],
       [2.37294160e-02, 9.73868600e-01, 2.40204410e-03],
       [2.05960570e-02, 8.97286700e-01, 8.21171800e-02],
       [2.20549650e-02, 2.57970040e-01, 7.19974940e-01],
       [3.79307130e-03, 5.30473850e-02, 9.43159500e-01],
       [8.32770900e-01, 4.40549630e-02, 1.23174110e-01],
       [2.72915100e-01, 3.47662660e-01, 3.79422300e-01],
       [2.23826620e-01, 7.47172500e-01, 2.90008620e-02],
       [9.85465500e-04, 2.10302550e-01, 7.88711970e-01],
       [8.70288200e-03, 5.44080600e-02, 9.36889100e-01],
       [1.87536720e-01, 5.78544700e-01, 2.33918650e-01],
       [1.74566960e-03, 2.52782850e-02, 9.72976000e-01],
       [8.60968770e-01, 1.06446830e-01, 3.25843800e-02],
       [3.62345130e-02, 9.46887850e-01, 1.68777020e-02],
       [2.53141580e-02, 6.95132300e-01, 2.79553560e-01],
       [3.09931860e-02, 1.47556990e-01, 8.21449800e-01],
       [9.51052900e-01, 3.14109470e-02, 1.75361240e-02],
       [4.09228350e-03, 8.51971

In [20]:
## Retrieved from https://github.com/scikit-learn/scikit-learn/issues/3298
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelBinarizer

def rocauc_ovo(truth, pred, average="macro", multi_class="ovo"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   
    
    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [21]:
ovo1 = rocauc_ovo(y_test, y_prob, average="macro", multi_class="ovo")
ovo1

0.779854769012316

In [22]:
def rocauc_ovr(truth, pred, average="macro", multi_class="ovr"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   

    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [23]:
ovr1 = rocauc_ovr(y_test, y_prob, average="macro", multi_class="ovr")
ovr1

0.779854769012316

In [24]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state=234,
                                                    stratify=y)

In [25]:
dat2 = pd.DataFrame(X_test.iloc[:,0])
dat2['test'] = y_test

In [26]:
dat2

,id,test
165,NRS191,2
237,SR1287,0
243,SR3569,2
128,NRS100,0
107,NRS001,1
...,...,...
220,NRS265,1
233,NY439,2
68,CFBRSa05,0
175,NRS205,2


In [27]:
X_train = X_train.drop(['id'], axis=1)
X_test = X_test.drop(['id'], axis=1)

In [28]:
model2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(3, activation='softmax'),
])

In [31]:
model2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [32]:
model2.fit(X_train, y_train,
          batch_size=32, epochs=100,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 974us/step - loss: 11.0972 - accuracy: 0.2655 - val_loss: 7.2993 - val_accuracy: 0.3158
Epoch 2/100
177/177 [==============================] - 0s 161us/step - loss: 8.2942 - accuracy: 0.3051 - val_loss: 4.9131 - val_accuracy: 0.3421
Epoch 3/100
177/177 [==============================] - 0s 162us/step - loss: 5.4000 - accuracy: 0.3277 - val_loss: 2.6774 - val_accuracy: 0.3947
Epoch 4/100
177/177 [==============================] - 0s 178us/step - loss: 2.9504 - accuracy: 0.3277 - val_loss: 1.2709 - val_accuracy: 0.4474
Epoch 5/100
177/177 [==============================] - 0s 174us/step - loss: 1.8934 - accuracy: 0.4068 - val_loss: 1.1951 - val_accuracy: 0.4079
Epoch 6/100
177/177 [==============================] - 0s 159us/step - loss: 1.4566 - accuracy: 0.4407 - val_loss: 1.9150 - val_accuracy: 0.4474
Epoch 7/100
177/177 [==============================] - 0s 220us/step - loss: 1.2930 

Epoch 57/100
177/177 [==============================] - 0s 105us/step - loss: 0.4964 - accuracy: 0.8531 - val_loss: 0.9159 - val_accuracy: 0.5921
Epoch 58/100
177/177 [==============================] - 0s 92us/step - loss: 0.4889 - accuracy: 0.8418 - val_loss: 0.9693 - val_accuracy: 0.5921
Epoch 59/100
177/177 [==============================] - 0s 105us/step - loss: 0.4755 - accuracy: 0.8475 - val_loss: 0.9435 - val_accuracy: 0.6184
Epoch 60/100
177/177 [==============================] - 0s 117us/step - loss: 0.4766 - accuracy: 0.8701 - val_loss: 0.9344 - val_accuracy: 0.6184
Epoch 61/100
177/177 [==============================] - 0s 95us/step - loss: 0.4666 - accuracy: 0.8644 - val_loss: 0.9467 - val_accuracy: 0.6184
Epoch 62/100
177/177 [==============================] - 0s 98us/step - loss: 0.4641 - accuracy: 0.8588 - val_loss: 1.0258 - val_accuracy: 0.5921
Epoch 63/100
177/177 [==============================] - 0s 99us/step - loss: 0.4571 - accuracy: 0.8814 - val_loss: 0.9201 - val

In [65]:
acc_test2 = model2.evaluate(X_test, y_test)[1]
print('test accuracy: %.2f%%' % (acc_test2*100))

76/76 [==============================] - 0s 112us/step
test accuracy: 64.47%


In [33]:
pred2 = model2.predict_classes(X_test)
pred2

array([2, 0, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 2, 2, 1, 1, 2, 2, 2, 1, 1, 2,
       0, 2, 0, 1, 1, 0, 2, 0, 2, 2, 1, 0, 1, 1, 2, 0, 1, 2, 2, 2, 2, 1,
       1, 1, 0, 1, 2, 1, 2, 2, 2, 1, 1, 1, 1, 0, 1, 1, 2, 2, 2, 2, 1, 2,
       1, 2, 1, 2, 2, 1, 2, 0, 2, 0])

In [34]:
dat2['pred'] = pred2
dat2

,id,test,pred
165,NRS191,2,2
237,SR1287,0,0
243,SR3569,2,2
128,NRS100,0,1
107,NRS001,1,1
...,...,...,...
220,NRS265,1,1
233,NY439,2,2
68,CFBRSa05,0,0
175,NRS205,2,2


In [35]:
proba2 = model2.predict_proba(X_test)
dat_proba2 = pd.DataFrame(proba2)

In [36]:
dat_proba2

,0,1,2
0,0.074384,0.160355,0.765261
1,0.575869,0.423838,0.000294
2,0.206686,0.294262,0.499052
3,0.100446,0.820180,0.079374
4,0.163093,0.773945,0.062962
...,...,...,...
71,0.025601,0.687962,0.286437
72,0.161947,0.266501,0.571552
73,0.652983,0.254172,0.092845
74,0.000927,0.033760,0.965313


In [37]:
dat_proba2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba2.csv", index = False,
         header=None)

In [38]:
dat2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/2p40pST.csv", index = False,
         header=None)

In [69]:
hist2 = model2.fit(X_train, y_train,
          batch_size=32, epochs=100,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 176us/step - loss: 0.2759 - accuracy: 0.9435 - val_loss: 1.2241 - val_accuracy: 0.6184
Epoch 2/100
177/177 [==============================] - 0s 185us/step - loss: 0.2754 - accuracy: 0.9322 - val_loss: 1.0206 - val_accuracy: 0.6579
Epoch 3/100
177/177 [==============================] - 0s 255us/step - loss: 0.3745 - accuracy: 0.8927 - val_loss: 1.3395 - val_accuracy: 0.6316
Epoch 4/100
177/177 [==============================] - 0s 136us/step - loss: 0.2695 - accuracy: 0.9548 - val_loss: 1.0070 - val_accuracy: 0.6316
Epoch 5/100
177/177 [==============================] - 0s 150us/step - loss: 0.2615 - accuracy: 0.9492 - val_loss: 1.1151 - val_accuracy: 0.6579
Epoch 6/100
177/177 [==============================] - 0s 134us/step - loss: 0.2604 - accuracy: 0.9322 - val_loss: 1.1596 - val_accuracy: 0.6053
Epoch 7/100
177/177 [==============================] - 0s 170us/step - loss: 0.2479 -

Epoch 57/100
177/177 [==============================] - 0s 133us/step - loss: 0.1708 - accuracy: 0.9718 - val_loss: 1.2437 - val_accuracy: 0.6579
Epoch 58/100
177/177 [==============================] - 0s 125us/step - loss: 0.1646 - accuracy: 0.9661 - val_loss: 1.1902 - val_accuracy: 0.6579
Epoch 59/100
177/177 [==============================] - 0s 131us/step - loss: 0.1712 - accuracy: 0.9605 - val_loss: 1.2996 - val_accuracy: 0.6053
Epoch 60/100
177/177 [==============================] - 0s 132us/step - loss: 0.1741 - accuracy: 0.9492 - val_loss: 1.2368 - val_accuracy: 0.6316
Epoch 61/100
177/177 [==============================] - 0s 139us/step - loss: 0.1649 - accuracy: 0.9548 - val_loss: 1.1552 - val_accuracy: 0.6447
Epoch 62/100
177/177 [==============================] - 0s 177us/step - loss: 0.1618 - accuracy: 0.9661 - val_loss: 1.3421 - val_accuracy: 0.6316
Epoch 63/100
177/177 [==============================] - 0s 197us/step - loss: 0.1607 - accuracy: 0.9661 - val_loss: 1.2467 -

In [70]:
print('train accuracy: %.2f%%' % (np.mean(hist2.history['accuracy'])*100))

train accuracy: 95.46%


In [29]:
df_proba2 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_2.xlsx",
                        sheet_name=1,
                        index_col=None)

In [30]:
df_proba2

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,NRS210,2,0,0.999887,0.000112,8.851192e-07
1,p0017kpresabs_qual,Grady1,0,0,0.625329,0.369782,4.889404e-03
2,p0017kpresabs_qual,CFBRSa29,2,0,0.999098,0.000269,6.335156e-04
3,p0017kpresabs_qual,CFBRSa03,0,0,0.647338,0.331796,2.086646e-02
4,p0017kpresabs_qual,217,1,0,0.613342,0.381903,4.754707e-03
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,NRS265,1,1,0.025601,0.687962,2.864372e-01
604,p0040presabsSTCC_qual,NY439,2,2,0.161947,0.266501,5.715521e-01
605,p0040presabsSTCC_qual,CFBRSa05,0,0,0.652983,0.254172,9.284494e-02
606,p0040presabsSTCC_qual,NRS205,2,2,0.000927,0.033760,9.653131e-01


In [31]:
y_prob2 = df_proba2[df_proba2['phage']=='p0040presabsSTCC_qual'].iloc[:,-3:]
y_prob2 = y_prob2.to_numpy()
y_prob2

array([[7.43840000e-02, 1.60354990e-01, 7.65261050e-01],
       [5.75868960e-01, 4.23837500e-01, 2.93562500e-04],
       [2.06685980e-01, 2.94262170e-01, 4.99051800e-01],
       [1.00446410e-01, 8.20179800e-01, 7.93738000e-02],
       [1.63093390e-01, 7.73944560e-01, 6.29620000e-02],
       [1.15806780e-01, 7.69957070e-01, 1.14236120e-01],
       [5.42172700e-02, 9.22629800e-01, 2.31529850e-02],
       [2.17800420e-03, 4.38636870e-03, 9.93435600e-01],
       [5.26644480e-02, 3.76925770e-02, 9.09642930e-01],
       [1.02666944e-01, 7.95859300e-01, 1.01473846e-01],
       [3.14865080e-01, 6.53080640e-01, 3.20542570e-02],
       [6.90679700e-03, 7.29548450e-01, 2.63544700e-01],
       [1.06337580e-01, 2.74829700e-01, 6.18832770e-01],
       [3.07671900e-01, 1.50208520e-01, 5.42119500e-01],
       [3.31465000e-01, 4.43395530e-01, 2.25139450e-01],
       [8.92775000e-02, 5.85423600e-01, 3.25298900e-01],
       [2.55964930e-03, 5.61172240e-03, 9.91828600e-01],
       [2.42558480e-01, 2.66818

In [32]:
ovo2 = rocauc_ovo(y_test, y_prob2, average="macro", multi_class="ovo")
ovo2

0.7528018903746926

In [33]:
ovr2 = rocauc_ovr(y_test, y_prob2, average="macro", multi_class="ovr")
ovr2

0.7528018903746926

In [34]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state=345,
                                                    stratify=y)

In [35]:
dat3 = pd.DataFrame(X_test.iloc[:,0])
dat3['test'] = y_test

In [36]:
dat3

,id,test
165,NRS191,2
64,CFBREBSa135,0
121,NRS064,1
228,NY224,1
114,NRS035,1
...,...,...
16,BCH-SA-01,0
13,504,0
96,GA27,2
177,NRS209,1


In [37]:
X_train = X_train.drop(['id'], axis=1)
X_test = X_test.drop(['id'], axis=1)

In [38]:
model3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [44]:
model3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [45]:
model3.fit(X_train, y_train,
          batch_size=16, epochs=100,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 1ms/step - loss: 2.0879 - accuracy: 0.4068 - val_loss: 1.8626 - val_accuracy: 0.3026
Epoch 2/100
177/177 [==============================] - 0s 254us/step - loss: 1.0469 - accuracy: 0.4689 - val_loss: 1.1992 - val_accuracy: 0.3816
Epoch 3/100
177/177 [==============================] - 0s 258us/step - loss: 1.2567 - accuracy: 0.5254 - val_loss: 1.1634 - val_accuracy: 0.3947
Epoch 4/100
177/177 [==============================] - 0s 214us/step - loss: 1.2061 - accuracy: 0.5424 - val_loss: 1.3853 - val_accuracy: 0.4211
Epoch 5/100
177/177 [==============================] - 0s 217us/step - loss: 1.5367 - accuracy: 0.5085 - val_loss: 1.3924 - val_accuracy: 0.4211
Epoch 6/100
177/177 [==============================] - 0s 217us/step - loss: 1.5927 - accuracy: 0.5537 - val_loss: 2.0409 - val_accuracy: 0.3816
Epoch 7/100
177/177 [==============================] - 0s 208us/step - loss: 1.6523 - a

Epoch 57/100
177/177 [==============================] - 0s 176us/step - loss: 0.6251 - accuracy: 0.8023 - val_loss: 1.3753 - val_accuracy: 0.5526
Epoch 58/100
177/177 [==============================] - 0s 162us/step - loss: 0.6119 - accuracy: 0.7910 - val_loss: 1.2778 - val_accuracy: 0.5000
Epoch 59/100
177/177 [==============================] - 0s 275us/step - loss: 0.7384 - accuracy: 0.7966 - val_loss: 1.2939 - val_accuracy: 0.5789
Epoch 60/100
177/177 [==============================] - 0s 363us/step - loss: 0.6418 - accuracy: 0.8362 - val_loss: 1.2806 - val_accuracy: 0.5263
Epoch 61/100
177/177 [==============================] - 0s 358us/step - loss: 0.4708 - accuracy: 0.8531 - val_loss: 1.3276 - val_accuracy: 0.5921
Epoch 62/100
177/177 [==============================] - 0s 234us/step - loss: 0.5776 - accuracy: 0.8475 - val_loss: 1.3404 - val_accuracy: 0.5658
Epoch 63/100
177/177 [==============================] - 0s 382us/step - loss: 0.6107 - accuracy: 0.8531 - val_loss: 2.0003 -

In [101]:
acc_test3 = model3.evaluate(X_test, y_test)[1]
print('test accuracy: %.2f%%' % (acc_test3*100))

76/76 [==============================] - 0s 125us/step
test accuracy: 56.58%


In [46]:
pred3 = model3.predict_classes(X_test)
pred3

array([2, 0, 1, 1, 2, 1, 0, 1, 1, 1, 2, 0, 2, 1, 2, 1, 0, 0, 2, 0, 0, 1,
       2, 2, 2, 2, 2, 2, 0, 1, 2, 0, 1, 1, 2, 0, 0, 0, 0, 2, 0, 2, 1, 1,
       2, 0, 2, 0, 0, 1, 2, 2, 0, 0, 2, 2, 1, 1, 0, 0, 0, 1, 1, 1, 2, 0,
       0, 2, 2, 1, 1, 0, 1, 1, 0, 1])

In [47]:
dat3['pred'] = pred3
dat3

,id,test,pred
165,NRS191,2,2
64,CFBREBSa135,0,0
121,NRS064,1,1
228,NY224,1,1
114,NRS035,1,2
...,...,...,...
16,BCH-SA-01,0,0
13,504,0,1
96,GA27,2,1
177,NRS209,1,0


In [48]:
proba3 = model3.predict_proba(X_test)
dat_proba3 = pd.DataFrame(proba3)

In [49]:
dat_proba3

,0,1,2
0,0.045016,0.407758,0.547227
1,0.873840,0.126159,0.000001
2,0.152263,0.507359,0.340378
3,0.188533,0.611574,0.199892
4,0.221121,0.120028,0.658852
...,...,...,...
71,0.930541,0.063563,0.005897
72,0.044764,0.857755,0.097481
73,0.277946,0.538438,0.183617
74,0.421018,0.355939,0.223043


In [50]:
dat_proba3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba3.csv", index = False,
         header=None)

In [51]:
dat3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/3p40pST.csv", index = False,
         header=None)

In [105]:
hist3 = model3.fit(X_train, y_train,
          batch_size=16, epochs=100,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 244us/step - loss: 0.2279 - accuracy: 0.9322 - val_loss: 1.5155 - val_accuracy: 0.5789
Epoch 2/100
177/177 [==============================] - 0s 297us/step - loss: 0.2667 - accuracy: 0.9209 - val_loss: 1.5713 - val_accuracy: 0.5789
Epoch 3/100
177/177 [==============================] - 0s 245us/step - loss: 0.2626 - accuracy: 0.9209 - val_loss: 1.6000 - val_accuracy: 0.5526
Epoch 4/100
177/177 [==============================] - 0s 257us/step - loss: 0.2335 - accuracy: 0.9266 - val_loss: 1.5858 - val_accuracy: 0.5658
Epoch 5/100
177/177 [==============================] - 0s 205us/step - loss: 0.2772 - accuracy: 0.8983 - val_loss: 1.6683 - val_accuracy: 0.5789
Epoch 6/100
177/177 [==============================] - 0s 203us/step - loss: 0.3017 - accuracy: 0.9040 - val_loss: 1.6611 - val_accuracy: 0.5526
Epoch 7/100
177/177 [==============================] - 0s 201us/step - loss: 0.5454 -

Epoch 57/100
177/177 [==============================] - 0s 232us/step - loss: 0.2776 - accuracy: 0.9266 - val_loss: 2.0380 - val_accuracy: 0.5658
Epoch 58/100
177/177 [==============================] - 0s 248us/step - loss: 0.1961 - accuracy: 0.9153 - val_loss: 1.8605 - val_accuracy: 0.5526
Epoch 59/100
177/177 [==============================] - 0s 205us/step - loss: 0.1919 - accuracy: 0.9379 - val_loss: 1.8463 - val_accuracy: 0.5263
Epoch 60/100
177/177 [==============================] - 0s 167us/step - loss: 0.2452 - accuracy: 0.9661 - val_loss: 1.8740 - val_accuracy: 0.5395
Epoch 61/100
177/177 [==============================] - 0s 204us/step - loss: 0.1761 - accuracy: 0.9492 - val_loss: 1.9183 - val_accuracy: 0.5000
Epoch 62/100
177/177 [==============================] - 0s 162us/step - loss: 0.2160 - accuracy: 0.9492 - val_loss: 1.9003 - val_accuracy: 0.5263
Epoch 63/100
177/177 [==============================] - 0s 191us/step - loss: 0.2874 - accuracy: 0.9209 - val_loss: 1.9066 -

In [106]:
print('train accuracy: %.2f%%' % (np.mean(hist3.history['accuracy'])*100))

train accuracy: 93.91%


In [39]:
df_proba3 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_2.xlsx",
                        sheet_name=2,
                        index_col=None)

In [40]:
df_proba3

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,NY360,2,2,2.165526e-02,4.848140e-01,0.493531
1,p0017kpresabs_qual,EUH25,2,0,9.986388e-01,1.245148e-03,0.000116
2,p0017kpresabs_qual,EUH15,2,2,9.227520e-04,1.424882e-02,0.984828
3,p0017kpresabs_qual,NRS241,0,0,8.374333e-01,1.614128e-01,0.001154
4,p0017kpresabs_qual,SR2852,2,2,3.976981e-09,5.145955e-10,1.000000
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,BCH-SA-01,0,0,9.305407e-01,6.356251e-02,0.005897
604,p0040presabsSTCC_qual,504,0,1,4.476389e-02,8.577548e-01,0.097481
605,p0040presabsSTCC_qual,GA27,2,1,2.779456e-01,5.384378e-01,0.183617
606,p0040presabsSTCC_qual,NRS209,1,0,4.210180e-01,3.559393e-01,0.223043


In [41]:
y_prob3 = df_proba3[df_proba3['phage']=='p0040presabsSTCC_qual'].iloc[:,-3:]
y_prob3 = y_prob3.to_numpy()
y_prob3

array([[4.50156550e-02, 4.07757820e-01, 5.47226550e-01],
       [8.73840000e-01, 1.26158580e-01, 1.37086910e-06],
       [1.52262800e-01, 5.07358800e-01, 3.40378430e-01],
       [1.88533440e-01, 6.11574350e-01, 1.99892160e-01],
       [2.21120770e-01, 1.20027676e-01, 6.58851560e-01],
       [1.33512830e-02, 8.20513300e-01, 1.66135420e-01],
       [9.93191400e-01, 6.79963450e-03, 8.95339500e-06],
       [2.46647900e-01, 7.44919500e-01, 8.43269900e-03],
       [4.17147200e-03, 7.21055200e-01, 2.74773300e-01],
       [4.80132130e-03, 8.53080800e-01, 1.42117840e-01],
       [2.34693440e-01, 1.82161480e-02, 7.47090400e-01],
       [7.66563600e-01, 1.04251890e-01, 1.29184620e-01],
       [1.00096810e-02, 1.29414470e-01, 8.60575850e-01],
       [7.73045350e-04, 8.84358600e-01, 1.14868370e-01],
       [1.55933620e-01, 6.19436300e-02, 7.82122730e-01],
       [3.62336780e-04, 9.90075050e-01, 9.56265400e-03],
       [8.50974140e-01, 1.28803410e-02, 1.36145500e-01],
       [8.41443300e-01, 1.12916

In [42]:
ovo3 = rocauc_ovo(y_test, y_prob3, average="macro", multi_class="ovo")
ovo3

0.7187085396040204

In [43]:
ovr3 = rocauc_ovr(y_test, y_prob3, average="macro", multi_class="ovr")
ovr3

0.7187085396040204

In [44]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state=456,
                                                    stratify=y)

In [45]:
dat4 = pd.DataFrame(X_test.iloc[:,0])
dat4['test'] = y_test

In [46]:
dat4

,id,test
54,CFBREBSa121,1
28,BCH-SA-13,1
143,NRS148,2
73,CFBRSa23,2
57,CFBREBSa125,0
...,...,...
208,NRS247,1
183,NRS215,1
248,SR4152,2
114,NRS035,1


In [47]:
X_train = X_train.drop(['id'], axis=1)
X_test = X_test.drop(['id'], axis=1)

In [48]:
model4 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(3, activation='softmax'),
])

In [57]:
model4.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [58]:
model4.fit(X_train, y_train,
          batch_size=32, epochs=100,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 909us/step - loss: 3.6790 - accuracy: 0.3107 - val_loss: 4.0488 - val_accuracy: 0.3158
Epoch 2/100
177/177 [==============================] - 0s 121us/step - loss: 2.4450 - accuracy: 0.3390 - val_loss: 2.6795 - val_accuracy: 0.3816
Epoch 3/100
177/177 [==============================] - 0s 198us/step - loss: 1.9212 - accuracy: 0.3955 - val_loss: 2.2386 - val_accuracy: 0.3816
Epoch 4/100
177/177 [==============================] - 0s 163us/step - loss: 1.3075 - accuracy: 0.4068 - val_loss: 2.3380 - val_accuracy: 0.3947
Epoch 5/100
177/177 [==============================] - 0s 236us/step - loss: 1.2188 - accuracy: 0.4972 - val_loss: 1.9985 - val_accuracy: 0.4605
Epoch 6/100
177/177 [==============================] - 0s 233us/step - loss: 0.9939 - accuracy: 0.5424 - val_loss: 1.6562 - val_accuracy: 0.4605
Epoch 7/100
177/177 [==============================] - 0s 213us/step - loss: 0.9501 -

Epoch 57/100
177/177 [==============================] - 0s 145us/step - loss: 0.4877 - accuracy: 0.8192 - val_loss: 1.5163 - val_accuracy: 0.4605
Epoch 58/100
177/177 [==============================] - 0s 150us/step - loss: 0.4608 - accuracy: 0.8305 - val_loss: 1.2087 - val_accuracy: 0.4474
Epoch 59/100
177/177 [==============================] - 0s 141us/step - loss: 0.5310 - accuracy: 0.8249 - val_loss: 1.1988 - val_accuracy: 0.4605
Epoch 60/100
177/177 [==============================] - 0s 125us/step - loss: 0.5984 - accuracy: 0.8079 - val_loss: 2.0378 - val_accuracy: 0.4868
Epoch 61/100
177/177 [==============================] - 0s 160us/step - loss: 0.6259 - accuracy: 0.8249 - val_loss: 1.2315 - val_accuracy: 0.4737
Epoch 62/100
177/177 [==============================] - 0s 126us/step - loss: 0.4690 - accuracy: 0.8249 - val_loss: 1.3871 - val_accuracy: 0.4474
Epoch 63/100
177/177 [==============================] - 0s 125us/step - loss: 0.4409 - accuracy: 0.8588 - val_loss: 1.3567 -

In [135]:
acc_test4 = model4.evaluate(X_test, y_test)[1]
print('test accuracy: %.2f%%' % (acc_test4*100))

76/76 [==============================] - 0s 67us/step
test accuracy: 55.26%


In [59]:
pred4 = model4.predict_classes(X_test)
pred4

array([0, 0, 2, 2, 0, 0, 2, 0, 2, 1, 2, 2, 0, 1, 0, 1, 2, 0, 1, 2, 2, 1,
       0, 2, 2, 2, 1, 0, 2, 2, 0, 2, 2, 2, 0, 1, 0, 0, 2, 2, 1, 0, 2, 1,
       0, 1, 0, 1, 1, 0, 0, 2, 1, 1, 1, 1, 1, 1, 2, 1, 0, 1, 0, 2, 2, 1,
       1, 1, 1, 1, 1, 2, 2, 2, 1, 2])

In [60]:
dat4['pred'] = pred4
dat4

,id,test,pred
54,CFBREBSa121,1,0
28,BCH-SA-13,1,0
143,NRS148,2,2
73,CFBRSa23,2,2
57,CFBREBSa125,0,0
...,...,...,...
208,NRS247,1,2
183,NRS215,1,2
248,SR4152,2,2
114,NRS035,1,1


In [61]:
proba4 = model4.predict_proba(X_test)
dat_proba4 = pd.DataFrame(proba4)

In [62]:
dat_proba4

,0,1,2
0,0.948243,0.044020,0.007737
1,0.562979,0.390488,0.046532
2,0.000052,0.012425,0.987523
3,0.039255,0.300029,0.660716
4,0.926907,0.063109,0.009983
...,...,...,...
71,0.029574,0.385730,0.584696
72,0.003197,0.445057,0.551746
73,0.043194,0.028984,0.927822
74,0.004902,0.680427,0.314671


In [63]:
dat_proba4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba4.csv", index = False,
         header=None)

In [64]:
dat4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/4p40pST.csv", index = False,
         header=None)

In [139]:
hist4 = model4.fit(X_train, y_train,
          batch_size=32, epochs=100,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 139us/step - loss: 0.2495 - accuracy: 0.9322 - val_loss: 1.2903 - val_accuracy: 0.5132
Epoch 2/100
177/177 [==============================] - 0s 125us/step - loss: 0.2448 - accuracy: 0.9435 - val_loss: 1.2871 - val_accuracy: 0.5263
Epoch 3/100
177/177 [==============================] - 0s 193us/step - loss: 0.2421 - accuracy: 0.9379 - val_loss: 1.2827 - val_accuracy: 0.5395
Epoch 4/100
177/177 [==============================] - 0s 155us/step - loss: 0.2426 - accuracy: 0.9379 - val_loss: 1.2937 - val_accuracy: 0.5132
Epoch 5/100
177/177 [==============================] - 0s 142us/step - loss: 0.2456 - accuracy: 0.9322 - val_loss: 1.3298 - val_accuracy: 0.5263
Epoch 6/100
177/177 [==============================] - 0s 150us/step - loss: 0.2346 - accuracy: 0.9322 - val_loss: 1.3244 - val_accuracy: 0.5395
Epoch 7/100
177/177 [==============================] - 0s 124us/step - loss: 0.2398 -

Epoch 57/100
177/177 [==============================] - 0s 122us/step - loss: 0.1880 - accuracy: 0.9661 - val_loss: 1.6281 - val_accuracy: 0.5000
Epoch 58/100
177/177 [==============================] - 0s 169us/step - loss: 0.2002 - accuracy: 0.9661 - val_loss: 2.1486 - val_accuracy: 0.5000
Epoch 59/100
177/177 [==============================] - 0s 135us/step - loss: 0.2242 - accuracy: 0.9492 - val_loss: 1.5632 - val_accuracy: 0.5132
Epoch 60/100
177/177 [==============================] - 0s 159us/step - loss: 0.1818 - accuracy: 0.9605 - val_loss: 1.5838 - val_accuracy: 0.5000
Epoch 61/100
177/177 [==============================] - 0s 171us/step - loss: 0.1591 - accuracy: 0.9661 - val_loss: 1.7335 - val_accuracy: 0.5132
Epoch 62/100
177/177 [==============================] - 0s 121us/step - loss: 0.1584 - accuracy: 0.9605 - val_loss: 1.6705 - val_accuracy: 0.5132
Epoch 63/100
177/177 [==============================] - 0s 317us/step - loss: 0.1470 - accuracy: 0.9718 - val_loss: 1.6382 -

In [140]:
print('train accuracy: %.2f%%' % (np.mean(hist4.history['accuracy'])*100))

train accuracy: 96.52%


In [49]:
df_proba4 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_2.xlsx",
                        sheet_name=3,
                        index_col=None)

In [50]:
df_proba4

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,SR1129,2,2,4.821690e-07,0.000001,9.999983e-01
1,p0017kpresabs_qual,CA105,2,2,2.652370e-03,0.000049,9.972990e-01
2,p0017kpresabs_qual,NRS175,1,1,1.198157e-02,0.988018,5.232074e-09
3,p0017kpresabs_qual,EUH25,2,1,1.388957e-01,0.735954,1.251503e-01
4,p0017kpresabs_qual,NRS070,2,1,7.231966e-03,0.810317,1.824512e-01
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,NRS247,1,2,2.957360e-02,0.385730,5.846961e-01
604,p0040presabsSTCC_qual,NRS215,1,2,3.196635e-03,0.445057,5.517461e-01
605,p0040presabsSTCC_qual,SR4152,2,2,4.319404e-02,0.028984,9.278219e-01
606,p0040presabsSTCC_qual,NRS035,1,1,4.902312e-03,0.680427,3.146706e-01


In [51]:
y_prob4 = df_proba4[df_proba4['phage']=='p0040presabsSTCC_qual'].iloc[:,-3:]
y_prob4 = y_prob4.to_numpy()
y_prob4

array([[9.48242660e-01, 4.40203060e-02, 7.73694370e-03],
       [5.62979200e-01, 3.90488420e-01, 4.65323850e-02],
       [5.24542340e-05, 1.24247240e-02, 9.87522800e-01],
       [3.92548900e-02, 3.00029250e-01, 6.60715900e-01],
       [9.26907360e-01, 6.31094800e-02, 9.98317900e-03],
       [8.29304700e-01, 1.40034930e-01, 3.06603090e-02],
       [4.29202300e-02, 6.74751400e-02, 8.89604600e-01],
       [9.54042850e-01, 3.26463700e-02, 1.33106920e-02],
       [4.61252300e-02, 1.82908760e-01, 7.70966000e-01],
       [1.02906720e-01, 4.93208940e-01, 4.03884350e-01],
       [9.12523100e-03, 6.08816520e-02, 9.29993150e-01],
       [3.35121300e-03, 1.36009580e-02, 9.83047840e-01],
       [8.46580450e-01, 1.45369840e-01, 8.04969100e-03],
       [3.06489170e-01, 6.50479850e-01, 4.30308950e-02],
       [9.00800900e-01, 8.67017400e-02, 1.24974090e-02],
       [6.83358730e-03, 9.80030100e-01, 1.31362940e-02],
       [2.09346340e-01, 6.95708400e-02, 7.21082800e-01],
       [6.94021460e-01, 2.75985

In [52]:
ovo4 = rocauc_ovo(y_test, y_prob4, average="macro", multi_class="ovo")
ovo4

0.6696745988881343

In [53]:
ovr4 = rocauc_ovr(y_test, y_prob4, average="macro", multi_class="ovr")
ovr4

0.6696745988881343

In [54]:
ovos = [ovo1, ovo2, ovo3, ovo4]
np.mean(ovos)

0.7302599494697908

In [55]:
np.std(ovos)

0.04114548881774576

In [56]:
ovrs = [ovr1, ovr2, ovr3, ovr4]
np.mean(ovrs)

0.7302599494697908

In [57]:
np.std(ovrs)

0.04114548881774576

In [141]:
accs = [acc_test1, acc_test2, acc_test3, acc_test4]

In [142]:
mean = np.mean(accs)
print('test accuracy mean: %.2f%%' % (mean*100))

test accuracy mean: 59.54%


In [143]:
std = np.std(accs)
print('test accuracy standard deviation:', std)

test accuracy standard deviation: 0.0376497545788795


In [144]:
accs_train = [np.mean(hist1.history['accuracy']), np.mean(hist2.history['accuracy']), np.mean(hist3.history['accuracy']),
             np.mean(hist4.history['accuracy'])]

In [145]:
mean_train = np.mean(accs_train)
print('train accuracy mean: %.2f%%' % (mean_train*100))

train accuracy mean: 95.13%


In [146]:
std_train = np.std(accs_train)
print('train accuracy standard deviation:', std_train)

train accuracy standard deviation: 0.009737109


In [147]:
############ Feature selection using lasso ##########

In [58]:
## Retrieved from https://towardsdatascience.com/feature-selection-using-regularisation-a3678b71e499
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
import numpy as np

In [59]:
selection = SelectFromModel(LogisticRegression(C=1, penalty='l1', solver='liblinear'))
selection.fit(X.loc[:, X.columns != 'id'], y)

SelectFromModel(estimator=LogisticRegression(C=1, class_weight=None, dual=False,
                                             fit_intercept=True,
                                             intercept_scaling=1, l1_ratio=None,
                                             max_iter=100, multi_class='auto',
                                             n_jobs=None, penalty='l1',
                                             random_state=None,
                                             solver='liblinear', tol=0.0001,
                                             verbose=0, warm_start=False),
                max_features=None, norm_order=1, prefit=False, threshold=None)

In [60]:
names = np.array(df_clean.columns).tolist()
names.remove('pheno')

In [61]:
X_train_features = np.vstack((names, X.loc[:, X.columns != 'id']))
X_train_features = pd.DataFrame(X_train_features)

In [62]:
sel_features = X_train_features.columns[(selection.get_support())]
print('total features: {}'.format((X_train_features.shape[1])))
print('selected features: {}'.format(len(sel_features)))

total features: 351
selected features: 121


In [63]:
cols = sel_features.values
cols.reshape((1, -1))

array([[  0,  12,  41,  44,  56,  62,  65,  68,  70,  71,  75,  76,  81,
         82,  83,  87,  88,  93,  94, 113, 116, 125, 126, 132, 134, 135,
        146, 148, 153, 164, 174, 180, 183, 184, 185, 186, 188, 189, 191,
        192, 193, 196, 197, 198, 200, 201, 202, 203, 206, 207, 210, 211,
        213, 215, 216, 217, 219, 220, 221, 225, 227, 228, 230, 231, 232,
        234, 236, 237, 239, 241, 244, 246, 247, 248, 251, 252, 253, 254,
        255, 263, 266, 269, 271, 273, 274, 279, 281, 284, 288, 291, 294,
        295, 298, 299, 301, 303, 304, 306, 308, 309, 311, 312, 313, 316,
        318, 319, 321, 323, 324, 325, 326, 329, 330, 332, 333, 334, 340,
        342, 345, 349, 350]])

In [64]:
names_arr = np.array(names)
names_arr[cols]

array(['TTTTTATATTGCGAAAAATAATTGGCGAACGAGGTAACTGGATACCTCATCCGCCAATTAAAATTTGTTAATTTAATAATTAAATATAAAGACGATTTAT',
       'TTTCACTGCCTGT', 'TGTCTGATTTTTT',
       'TGGCTTAGGCTATAAATTGGAAAGACCTAAAGAACAATGATGAAGTTTCACCACCGCTTAATGTTATTGATAAGCAGTATATTAATTATCAGTTTTGTTA',
       'TGAACGTTATCT', 'TCGTGTCTGT', 'TCGCTGAAATATT',
       'TCGCTGAAATATTTGCGACAT', 'TCATCAAACTTT', 'TCAGTAGAGAT',
       'TCACACCGCCT', 'TCACACCGCCTAT', 'TAGTCATACAAT', 'TAGCTAAATCC',
       'TAGATTCAAATAT', 'TAATGGTAGTAGATAATTTTTC', 'TAATCTTGTTGTT',
       'TAAATTCCATAC',
       'TAAATCGTCTTTATATTTAATTATTAAATTAACAAATTTTAATTGGCGGATGAGGTATCCAGTTACCTCGTTCGCCAATTATTTTTCGCAATATAAAAAG',
       'GTGGCTTAGGCTATAAATTGGAAAGACCTAAAGAACAATGATGAAGTTTCACCACCGCTTAATGTTATTGATAAGCAGTATATTAATTATCAGTTTTGTT',
       'GTGAATTCATG', 'GGAGGATGAG', 'GGAGGATGAGG', 'GCTAATTATGTTTCTGGATT',
       'GAGGAGGATGAGG', 'GAGGAAGCAGAT', 'CTGTTTAATGATT', 'CTAATCCTTCAAT',
       'CGCTGAAATATTTGCG', 'ATGTCGCAAATATTTCAGCGACTTGT',
       'ACTTTTTATATTGCGAAAAATAAT

In [65]:
###### keep selected variables as a new dataframe
df_sel = df_clean.loc[:,names_arr[cols]].copy()
df_sel['pheno'] = df_clean['pheno']

In [66]:
df_sel['strain'] = X.iloc[:,0]

In [67]:
df_sel

,TTTTTATATTGCGAAAAATAATTGGCGAACGAGGTAACTGGATACCTCATCCGCCAATTAAAATTTGTTAATTTAATAATTAAATATAAAGACGATTTAT,TTTCACTGCCTGT,TGTCTGATTTTTT,TGGCTTAGGCTATAAATTGGAAAGACCTAAAGAACAATGATGAAGTTTCACCACCGCTTAATGTTATTGATAAGCAGTATATTAATTATCAGTTTTGTTA,TGAACGTTATCT,TCGTGTCTGT,TCGCTGAAATATT,TCGCTGAAATATTTGCGACAT,TCATCAAACTTT,TCAGTAGAGAT,...,group_861,group_862,group_210,group_2236,group_8800,group_8093,ST,CC,pheno,strain
0,1,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,5,5,2,107
1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,8,8,0,109
2,0,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,5,5,2,115
3,0,1,1,0,1,1,1,1,1,1,...,0,0,0,0,0,0,5,5,2,120335
4,0,1,1,0,1,1,1,1,1,1,...,0,0,0,0,0,0,5,5,2,120337
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,0,1,1,0,1,1,1,1,1,1,...,0,0,0,0,0,0,5,5,2,SR4152
249,0,1,1,0,1,1,1,1,1,1,...,0,0,0,0,0,0,3812,5,1,SR4153
250,0,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,5,5,1,SR4155
251,1,0,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,5,5,2,SR4156


In [68]:
X_sel = df_sel.loc[:, df_sel.columns != 'pheno']
y_sel = df_sel['pheno']
print(X_sel.shape, y_sel.shape, df_sel.shape)

(253, 122) (253,) (253, 123)


In [69]:
df_sel['pheno'].value_counts()

2    95
1    94
0    64
Name: pheno, dtype: int64

In [70]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train, X_sel_test, y_sel_train, y_sel_test = train_test_split(X_sel, y_sel,
                                                    test_size = 0.3,
                                                    random_state=567,
                                                    stratify=y_sel)

In [71]:
dat5 = pd.DataFrame(X_sel_test.iloc[:,-1])
dat5['test'] = y_sel_test

In [72]:
dat5

,strain,test
251,SR4156,2
17,BCH-SA-02,0
158,NRS180,2
232,NY417,1
47,CFBREBSa110,2
...,...,...
62,CFBREBSa131,2
138,NRS112,2
21,BCH-SA-06,0
15,834N,2


In [73]:
X_sel_train = X_sel_train.drop(['strain'], axis=1)
X_sel_test = X_sel_test.drop(['strain'], axis=1)

In [81]:
#### neural network on over-sampling data
model_sel = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train.shape[1],)),
    Dense(3, activation='softmax'),
])

In [82]:
model_sel.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [83]:
model_sel.fit(X_sel_train, y_sel_train,
          batch_size=64, epochs=1000,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 671us/step - loss: 16.6302 - accuracy: 0.2373 - val_loss: 7.0770 - val_accuracy: 0.2105
Epoch 2/1000
177/177 [==============================] - 0s 81us/step - loss: 14.7159 - accuracy: 0.2768 - val_loss: 6.1042 - val_accuracy: 0.2500
Epoch 3/1000
177/177 [==============================] - 0s 98us/step - loss: 12.8403 - accuracy: 0.2881 - val_loss: 5.2717 - val_accuracy: 0.2237
Epoch 4/1000
177/177 [==============================] - 0s 105us/step - loss: 11.2429 - accuracy: 0.3107 - val_loss: 4.5567 - val_accuracy: 0.1842
Epoch 5/1000
177/177 [==============================] - 0s 117us/step - loss: 9.2450 - accuracy: 0.2938 - val_loss: 3.8828 - val_accuracy: 0.3684
Epoch 6/1000
177/177 [==============================] - 0s 101us/step - loss: 7.8674 - accuracy: 0.3503 - val_loss: 3.1239 - val_accuracy: 0.3421
Epoch 7/1000
177/177 [==============================] - 0s 101us/step - loss:

Epoch 57/1000
177/177 [==============================] - 0s 115us/step - loss: 0.6710 - accuracy: 0.8079 - val_loss: 1.0933 - val_accuracy: 0.4737
Epoch 58/1000
177/177 [==============================] - 0s 81us/step - loss: 0.6564 - accuracy: 0.7853 - val_loss: 1.0101 - val_accuracy: 0.4605
Epoch 59/1000
177/177 [==============================] - 0s 95us/step - loss: 0.6303 - accuracy: 0.8079 - val_loss: 1.0139 - val_accuracy: 0.4737
Epoch 60/1000
177/177 [==============================] - 0s 177us/step - loss: 0.6241 - accuracy: 0.8023 - val_loss: 1.0101 - val_accuracy: 0.4868
Epoch 61/1000
177/177 [==============================] - 0s 88us/step - loss: 0.6074 - accuracy: 0.8079 - val_loss: 1.0182 - val_accuracy: 0.4605
Epoch 62/1000
177/177 [==============================] - 0s 79us/step - loss: 0.6340 - accuracy: 0.8079 - val_loss: 1.0196 - val_accuracy: 0.4605
Epoch 63/1000
177/177 [==============================] - 0s 78us/step - loss: 0.6116 - accuracy: 0.8079 - val_loss: 1.0899

177/177 [==============================] - 0s 65us/step - loss: 0.4559 - accuracy: 0.8757 - val_loss: 1.2454 - val_accuracy: 0.4868
Epoch 168/1000
177/177 [==============================] - 0s 106us/step - loss: 0.4520 - accuracy: 0.9153 - val_loss: 1.2049 - val_accuracy: 0.5000
Epoch 169/1000
177/177 [==============================] - 0s 213us/step - loss: 0.3444 - accuracy: 0.9209 - val_loss: 1.0781 - val_accuracy: 0.4605
Epoch 170/1000
177/177 [==============================] - 0s 146us/step - loss: 0.3912 - accuracy: 0.9096 - val_loss: 1.0883 - val_accuracy: 0.4868
Epoch 171/1000
177/177 [==============================] - 0s 170us/step - loss: 0.3114 - accuracy: 0.9322 - val_loss: 1.1617 - val_accuracy: 0.5000
Epoch 172/1000
177/177 [==============================] - 0s 109us/step - loss: 0.3692 - accuracy: 0.9209 - val_loss: 1.1750 - val_accuracy: 0.4868
Epoch 173/1000
177/177 [==============================] - 0s 132us/step - loss: 0.3618 - accuracy: 0.9153 - val_loss: 1.1016 - v

Epoch 278/1000
177/177 [==============================] - 0s 71us/step - loss: 0.2172 - accuracy: 0.9548 - val_loss: 1.1782 - val_accuracy: 0.4868
Epoch 279/1000
177/177 [==============================] - 0s 69us/step - loss: 0.2190 - accuracy: 0.9548 - val_loss: 1.1910 - val_accuracy: 0.4737
Epoch 280/1000
177/177 [==============================] - 0s 80us/step - loss: 0.2177 - accuracy: 0.9435 - val_loss: 1.1796 - val_accuracy: 0.4868
Epoch 281/1000
177/177 [==============================] - 0s 61us/step - loss: 0.2153 - accuracy: 0.9492 - val_loss: 1.1740 - val_accuracy: 0.4737
Epoch 282/1000
177/177 [==============================] - 0s 65us/step - loss: 0.2002 - accuracy: 0.9605 - val_loss: 1.1985 - val_accuracy: 0.4737
Epoch 283/1000
177/177 [==============================] - 0s 66us/step - loss: 0.2234 - accuracy: 0.9605 - val_loss: 1.1870 - val_accuracy: 0.4737
Epoch 284/1000
177/177 [==============================] - 0s 77us/step - loss: 0.2259 - accuracy: 0.9492 - val_loss: 1

Epoch 334/1000
177/177 [==============================] - 0s 60us/step - loss: 0.2256 - accuracy: 0.9435 - val_loss: 1.2397 - val_accuracy: 0.4868
Epoch 335/1000
177/177 [==============================] - 0s 52us/step - loss: 0.2591 - accuracy: 0.9435 - val_loss: 1.2848 - val_accuracy: 0.4868
Epoch 336/1000
177/177 [==============================] - 0s 57us/step - loss: 0.2046 - accuracy: 0.9492 - val_loss: 1.2030 - val_accuracy: 0.4605
Epoch 337/1000
177/177 [==============================] - 0s 67us/step - loss: 0.2994 - accuracy: 0.9209 - val_loss: 1.2108 - val_accuracy: 0.5132
Epoch 338/1000
177/177 [==============================] - 0s 52us/step - loss: 0.1804 - accuracy: 0.9548 - val_loss: 1.3052 - val_accuracy: 0.5263
Epoch 339/1000
177/177 [==============================] - 0s 57us/step - loss: 0.2170 - accuracy: 0.9492 - val_loss: 1.2943 - val_accuracy: 0.5263
Epoch 340/1000
177/177 [==============================] - 0s 61us/step - loss: 0.1735 - accuracy: 0.9492 - val_loss: 1

...

Epoch 612/1000
177/177 [==============================] - 0s 66us/step - loss: 0.0789 - accuracy: 0.9831 - val_loss: 1.5502 - val_accuracy: 0.5132
Epoch 613/1000
177/177 [==============================] - 0s 71us/step - loss: 0.0884 - accuracy: 0.9774 - val_loss: 1.5383 - val_accuracy: 0.5132
Epoch 614/1000
177/177 [==============================] - 0s 67us/step - loss: 0.0850 - accuracy: 0.9831 - val_loss: 1.5518 - val_accuracy: 0.5132
Epoch 615/1000
177/177 [==============================] - 0s 48us/step - loss: 0.1024 - accuracy: 0.9831 - val_loss: 1.5443 - val_accuracy: 0.5132
Epoch 616/1000
177/177 [==============================] - 0s 79us/step - loss: 0.1073 - accuracy: 0.9831 - val_loss: 1.5228 - val_accuracy: 0.5000
Epoch 617/1000
177/177 [==============================] - 0s 70us/step - loss: 0.0760 - accuracy: 0.9887 - val_loss: 1.5669 - val_accuracy: 0.5132
Epoch 618/1000
177/177 [==============================] - ETA: 0s - loss: 0.1421 - accuracy: 0.95 - 0s 75us/step - los

Epoch 723/1000
177/177 [==============================] - 0s 64us/step - loss: 0.0769 - accuracy: 0.9831 - val_loss: 1.8070 - val_accuracy: 0.4868
Epoch 724/1000
177/177 [==============================] - 0s 66us/step - loss: 0.1834 - accuracy: 0.9718 - val_loss: 1.7152 - val_accuracy: 0.5132
Epoch 725/1000
177/177 [==============================] - 0s 71us/step - loss: 0.1490 - accuracy: 0.9661 - val_loss: 1.6346 - val_accuracy: 0.5132
Epoch 726/1000
177/177 [==============================] - 0s 71us/step - loss: 0.1816 - accuracy: 0.9661 - val_loss: 1.7536 - val_accuracy: 0.5263
Epoch 727/1000
177/177 [==============================] - 0s 70us/step - loss: 0.1450 - accuracy: 0.9774 - val_loss: 1.8421 - val_accuracy: 0.5000
Epoch 728/1000
177/177 [==============================] - 0s 67us/step - loss: 0.2012 - accuracy: 0.9661 - val_loss: 1.7032 - val_accuracy: 0.5000
Epoch 729/1000
177/177 [==============================] - 0s 66us/step - loss: 0.3286 - accuracy: 0.9492 - val_loss: 1

Epoch 779/1000
177/177 [==============================] - 0s 67us/step - loss: 0.0760 - accuracy: 0.9831 - val_loss: 1.6886 - val_accuracy: 0.5132
Epoch 780/1000
177/177 [==============================] - 0s 73us/step - loss: 0.0649 - accuracy: 0.9831 - val_loss: 1.7226 - val_accuracy: 0.5132
Epoch 781/1000
177/177 [==============================] - 0s 65us/step - loss: 0.0574 - accuracy: 0.9944 - val_loss: 1.8351 - val_accuracy: 0.5263
Epoch 782/1000
177/177 [==============================] - 0s 61us/step - loss: 0.1090 - accuracy: 0.9887 - val_loss: 1.7545 - val_accuracy: 0.5132
Epoch 783/1000
177/177 [==============================] - 0s 65us/step - loss: 0.0561 - accuracy: 0.9944 - val_loss: 1.7051 - val_accuracy: 0.5263
Epoch 784/1000
177/177 [==============================] - 0s 61us/step - loss: 0.0604 - accuracy: 0.9887 - val_loss: 1.7032 - val_accuracy: 0.5263
Epoch 785/1000
177/177 [==============================] - 0s 66us/step - loss: 0.0584 - accuracy: 0.9944 - val_loss: 1

Epoch 835/1000
177/177 [==============================] - 0s 62us/step - loss: 0.0719 - accuracy: 0.9831 - val_loss: 1.7993 - val_accuracy: 0.5395
Epoch 836/1000
177/177 [==============================] - 0s 58us/step - loss: 0.1030 - accuracy: 0.9831 - val_loss: 1.8610 - val_accuracy: 0.5263
Epoch 837/1000
177/177 [==============================] - 0s 73us/step - loss: 0.0576 - accuracy: 0.9887 - val_loss: 1.7614 - val_accuracy: 0.5395
Epoch 838/1000
177/177 [==============================] - 0s 73us/step - loss: 0.0768 - accuracy: 0.9831 - val_loss: 1.7896 - val_accuracy: 0.5395
Epoch 839/1000
177/177 [==============================] - 0s 57us/step - loss: 0.0541 - accuracy: 0.9944 - val_loss: 1.7964 - val_accuracy: 0.5395
Epoch 840/1000
177/177 [==============================] - 0s 56us/step - loss: 0.0578 - accuracy: 0.9887 - val_loss: 1.7733 - val_accuracy: 0.5132
Epoch 841/1000
177/177 [==============================] - 0s 64us/step - loss: 0.0738 - accuracy: 0.9887 - val_loss: 1

Epoch 946/1000
177/177 [==============================] - 0s 85us/step - loss: 0.0712 - accuracy: 0.9887 - val_loss: 2.0811 - val_accuracy: 0.5395
Epoch 947/1000
177/177 [==============================] - 0s 70us/step - loss: 0.2373 - accuracy: 0.9605 - val_loss: 1.9118 - val_accuracy: 0.5263
Epoch 948/1000
177/177 [==============================] - 0s 97us/step - loss: 0.1447 - accuracy: 0.9661 - val_loss: 1.8186 - val_accuracy: 0.5263
Epoch 949/1000
177/177 [==============================] - 0s 88us/step - loss: 0.1666 - accuracy: 0.9774 - val_loss: 1.9808 - val_accuracy: 0.5132
Epoch 950/1000
177/177 [==============================] - 0s 72us/step - loss: 0.0668 - accuracy: 0.9887 - val_loss: 1.9842 - val_accuracy: 0.5263
Epoch 951/1000
177/177 [==============================] - 0s 69us/step - loss: 0.1368 - accuracy: 0.9831 - val_loss: 1.9100 - val_accuracy: 0.5395
Epoch 952/1000
177/177 [==============================] - 0s 67us/step - loss: 0.1639 - accuracy: 0.9661 - val_loss: 1

In [211]:
acc_test_sel = model_sel.evaluate(X_sel_test, y_sel_test)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_sel*100))

76/76 [==============================] - 0s 111us/step
over-sampling test accuracy: 52.63%


In [84]:
pred5 = model_sel.predict_classes(X_sel_test)
pred5

array([2, 0, 2, 1, 2, 2, 2, 2, 2, 1, 1, 2, 2, 1, 1, 2, 0, 2, 0, 2, 1, 2,
       2, 0, 0, 1, 0, 1, 2, 0, 0, 2, 2, 1, 1, 0, 1, 2, 2, 1, 0, 0, 2, 2,
       1, 2, 2, 0, 2, 1, 1, 2, 0, 1, 1, 1, 1, 1, 1, 2, 2, 1, 0, 0, 2, 1,
       0, 1, 2, 2, 1, 1, 2, 0, 2, 2])

In [85]:
dat5['pred'] = pred5
dat5

,strain,test,pred
251,SR4156,2,2
17,BCH-SA-02,0,0
158,NRS180,2,2
232,NY417,1,1
47,CFBREBSa110,2,2
...,...,...,...
62,CFBREBSa131,2,1
138,NRS112,2,2
21,BCH-SA-06,0,0
15,834N,2,2


In [86]:
proba5 = model_sel.predict_proba(X_sel_test)
dat_proba5 = pd.DataFrame(proba5)

In [87]:
dat_proba5

,0,1,2
0,0.000752,0.002747,0.996501
1,0.897230,0.060890,0.041880
2,0.000070,0.047086,0.952844
3,0.288528,0.536828,0.174645
4,0.015682,0.005863,0.978455
...,...,...,...
71,0.006617,0.565843,0.427540
72,0.001056,0.045611,0.953333
73,0.874708,0.071162,0.054130
74,0.000003,0.000013,0.999985


In [88]:
dat_proba5.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba5.csv", index = False,
         header=None)

In [89]:
dat5.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/5p40pST.csv", index = False,
         header=None)

In [215]:
hist_sel = model_sel.fit(X_sel_train, y_sel_train,
          batch_size=64, epochs=1000,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 93us/step - loss: 0.0445 - accuracy: 0.9887 - val_loss: 2.1354 - val_accuracy: 0.5263
Epoch 2/1000
177/177 [==============================] - 0s 70us/step - loss: 0.0304 - accuracy: 0.9944 - val_loss: 2.1791 - val_accuracy: 0.5000
Epoch 3/1000
177/177 [==============================] - 0s 84us/step - loss: 0.0714 - accuracy: 0.9887 - val_loss: 2.1723 - val_accuracy: 0.5132
Epoch 4/1000
177/177 [==============================] - 0s 110us/step - loss: 0.0841 - accuracy: 0.9774 - val_loss: 2.1208 - val_accuracy: 0.5263
Epoch 5/1000
177/177 [==============================] - 0s 121us/step - loss: 0.0758 - accuracy: 0.9887 - val_loss: 2.1347 - val_accuracy: 0.5263
Epoch 6/1000
177/177 [==============================] - 0s 122us/step - loss: 0.0539 - accuracy: 0.9831 - val_loss: 2.2580 - val_accuracy: 0.5132
Epoch 7/1000
177/177 [==============================] - 0s 95us/step - loss: 0.088

177/177 [==============================] - 0s 59us/step - loss: 0.0244 - accuracy: 0.9944 - val_loss: 2.3139 - val_accuracy: 0.5132
Epoch 113/1000
177/177 [==============================] - 0s 53us/step - loss: 0.0243 - accuracy: 0.9944 - val_loss: 2.3048 - val_accuracy: 0.5263
Epoch 114/1000
177/177 [==============================] - 0s 64us/step - loss: 0.0241 - accuracy: 0.9944 - val_loss: 2.3053 - val_accuracy: 0.5263
Epoch 115/1000
177/177 [==============================] - 0s 64us/step - loss: 0.0241 - accuracy: 0.9944 - val_loss: 2.3111 - val_accuracy: 0.5263
Epoch 116/1000
177/177 [==============================] - 0s 66us/step - loss: 0.0247 - accuracy: 0.9944 - val_loss: 2.3081 - val_accuracy: 0.5263
Epoch 117/1000
177/177 [==============================] - 0s 90us/step - loss: 0.0252 - accuracy: 0.9944 - val_loss: 2.3034 - val_accuracy: 0.5263
Epoch 118/1000
177/177 [==============================] - 0s 75us/step - loss: 0.0246 - accuracy: 1.0000 - val_loss: 2.3151 - val_acc

Epoch 223/1000
177/177 [==============================] - 0s 65us/step - loss: 0.1969 - accuracy: 0.9831 - val_loss: 2.4336 - val_accuracy: 0.5132
Epoch 224/1000
177/177 [==============================] - 0s 56us/step - loss: 0.1117 - accuracy: 0.9831 - val_loss: 2.4052 - val_accuracy: 0.5132
Epoch 225/1000
177/177 [==============================] - 0s 55us/step - loss: 0.0445 - accuracy: 0.9831 - val_loss: 2.5071 - val_accuracy: 0.5000
Epoch 226/1000
177/177 [==============================] - 0s 51us/step - loss: 0.1614 - accuracy: 0.9661 - val_loss: 2.5845 - val_accuracy: 0.5132
Epoch 227/1000
177/177 [==============================] - 0s 54us/step - loss: 0.0670 - accuracy: 0.9831 - val_loss: 2.4216 - val_accuracy: 0.5395
Epoch 228/1000
177/177 [==============================] - 0s 62us/step - loss: 0.0977 - accuracy: 0.9887 - val_loss: 2.4246 - val_accuracy: 0.5263
Epoch 229/1000
177/177 [==============================] - 0s 62us/step - loss: 0.1214 - accuracy: 0.9831 - val_loss: 2

Epoch 334/1000
177/177 [==============================] - 0s 98us/step - loss: 0.0164 - accuracy: 1.0000 - val_loss: 2.5377 - val_accuracy: 0.5263
Epoch 335/1000
177/177 [==============================] - 0s 94us/step - loss: 0.0160 - accuracy: 1.0000 - val_loss: 2.5452 - val_accuracy: 0.5263
Epoch 336/1000
177/177 [==============================] - 0s 101us/step - loss: 0.0172 - accuracy: 1.0000 - val_loss: 2.5470 - val_accuracy: 0.5263
Epoch 337/1000
177/177 [==============================] - 0s 90us/step - loss: 0.0165 - accuracy: 1.0000 - val_loss: 2.5346 - val_accuracy: 0.5263
Epoch 338/1000
177/177 [==============================] - 0s 110us/step - loss: 0.0172 - accuracy: 1.0000 - val_loss: 2.5438 - val_accuracy: 0.5263
Epoch 339/1000
177/177 [==============================] - 0s 131us/step - loss: 0.0166 - accuracy: 1.0000 - val_loss: 2.5522 - val_accuracy: 0.5263
Epoch 340/1000
177/177 [==============================] - 0s 88us/step - loss: 0.0162 - accuracy: 1.0000 - val_loss

Epoch 390/1000
177/177 [==============================] - 0s 73us/step - loss: 0.0150 - accuracy: 1.0000 - val_loss: 2.6194 - val_accuracy: 0.5132
Epoch 391/1000
177/177 [==============================] - 0s 60us/step - loss: 0.0217 - accuracy: 0.9944 - val_loss: 2.5875 - val_accuracy: 0.5263
Epoch 392/1000
177/177 [==============================] - 0s 57us/step - loss: 0.0230 - accuracy: 0.9944 - val_loss: 2.5931 - val_accuracy: 0.5132
Epoch 393/1000
177/177 [==============================] - 0s 67us/step - loss: 0.0144 - accuracy: 1.0000 - val_loss: 2.6244 - val_accuracy: 0.5000
Epoch 394/1000
177/177 [==============================] - 0s 67us/step - loss: 0.0252 - accuracy: 0.9944 - val_loss: 2.5840 - val_accuracy: 0.5132
Epoch 395/1000
177/177 [==============================] - 0s 65us/step - loss: 0.0271 - accuracy: 0.9944 - val_loss: 2.5872 - val_accuracy: 0.5263
Epoch 396/1000
177/177 [==============================] - 0s 64us/step - loss: 0.0164 - accuracy: 1.0000 - val_loss: 2

...

Epoch 557/1000
177/177 [==============================] - 0s 95us/step - loss: 0.0383 - accuracy: 0.9831 - val_loss: 2.7438 - val_accuracy: 0.5132
Epoch 558/1000
177/177 [==============================] - 0s 56us/step - loss: 0.0545 - accuracy: 0.9887 - val_loss: 2.7115 - val_accuracy: 0.5132
Epoch 559/1000
177/177 [==============================] - 0s 72us/step - loss: 0.0645 - accuracy: 0.9887 - val_loss: 2.8187 - val_accuracy: 0.5000
Epoch 560/1000
177/177 [==============================] - 0s 107us/step - loss: 0.0654 - accuracy: 0.9831 - val_loss: 2.7716 - val_accuracy: 0.5132
Epoch 561/1000
177/177 [==============================] - 0s 62us/step - loss: 0.0784 - accuracy: 0.9944 - val_loss: 2.7242 - val_accuracy: 0.5132
Epoch 562/1000
177/177 [==============================] - 0s 57us/step - loss: 0.0582 - accuracy: 0.9887 - val_loss: 2.7843 - val_accuracy: 0.5000
Epoch 563/1000
177/177 [==============================] - 0s 65us/step - loss: 0.0924 - accuracy: 0.9831 - val_loss: 

Epoch 668/1000
177/177 [==============================] - 0s 106us/step - loss: 0.0098 - accuracy: 1.0000 - val_loss: 2.9035 - val_accuracy: 0.5000
Epoch 669/1000
177/177 [==============================] - 0s 105us/step - loss: 0.0098 - accuracy: 1.0000 - val_loss: 2.8924 - val_accuracy: 0.5132
Epoch 670/1000
177/177 [==============================] - 0s 62us/step - loss: 0.0099 - accuracy: 1.0000 - val_loss: 2.8930 - val_accuracy: 0.5132
Epoch 671/1000
177/177 [==============================] - 0s 56us/step - loss: 0.0098 - accuracy: 1.0000 - val_loss: 2.8907 - val_accuracy: 0.5132
Epoch 672/1000
177/177 [==============================] - 0s 62us/step - loss: 0.0097 - accuracy: 1.0000 - val_loss: 2.8936 - val_accuracy: 0.5132
Epoch 673/1000
177/177 [==============================] - 0s 61us/step - loss: 0.0096 - accuracy: 1.0000 - val_loss: 2.8966 - val_accuracy: 0.5132
Epoch 674/1000
177/177 [==============================] - 0s 58us/step - loss: 0.0098 - accuracy: 1.0000 - val_loss:

Epoch 779/1000
177/177 [==============================] - 0s 81us/step - loss: 0.0082 - accuracy: 1.0000 - val_loss: 2.9755 - val_accuracy: 0.5132
Epoch 780/1000
177/177 [==============================] - 0s 62us/step - loss: 0.0082 - accuracy: 1.0000 - val_loss: 2.9737 - val_accuracy: 0.5132
Epoch 781/1000
177/177 [==============================] - 0s 87us/step - loss: 0.0081 - accuracy: 1.0000 - val_loss: 2.9727 - val_accuracy: 0.5263
Epoch 782/1000
177/177 [==============================] - 0s 77us/step - loss: 0.0082 - accuracy: 1.0000 - val_loss: 2.9709 - val_accuracy: 0.5263
Epoch 783/1000
177/177 [==============================] - 0s 87us/step - loss: 0.0081 - accuracy: 1.0000 - val_loss: 2.9655 - val_accuracy: 0.5263
Epoch 784/1000
177/177 [==============================] - 0s 130us/step - loss: 0.0084 - accuracy: 1.0000 - val_loss: 2.9619 - val_accuracy: 0.5263
Epoch 785/1000
177/177 [==============================] - 0s 166us/step - loss: 0.0082 - accuracy: 1.0000 - val_loss:

Epoch 835/1000
177/177 [==============================] - 0s 78us/step - loss: 0.0076 - accuracy: 1.0000 - val_loss: 3.0045 - val_accuracy: 0.5263
Epoch 836/1000
177/177 [==============================] - 0s 130us/step - loss: 0.0077 - accuracy: 1.0000 - val_loss: 3.0055 - val_accuracy: 0.5263
Epoch 837/1000
177/177 [==============================] - 0s 134us/step - loss: 0.0075 - accuracy: 1.0000 - val_loss: 3.0149 - val_accuracy: 0.5132
Epoch 838/1000
177/177 [==============================] - 0s 181us/step - loss: 0.0076 - accuracy: 1.0000 - val_loss: 3.0219 - val_accuracy: 0.5132
Epoch 839/1000
177/177 [==============================] - 0s 138us/step - loss: 0.0077 - accuracy: 1.0000 - val_loss: 3.0150 - val_accuracy: 0.5263
Epoch 840/1000
177/177 [==============================] - 0s 104us/step - loss: 0.0078 - accuracy: 1.0000 - val_loss: 3.0082 - val_accuracy: 0.5263
Epoch 841/1000
177/177 [==============================] - 0s 105us/step - loss: 0.0075 - accuracy: 1.0000 - val_l

Epoch 946/1000
177/177 [==============================] - 0s 68us/step - loss: 0.0064 - accuracy: 1.0000 - val_loss: 3.0808 - val_accuracy: 0.5263
Epoch 947/1000
177/177 [==============================] - 0s 68us/step - loss: 0.0064 - accuracy: 1.0000 - val_loss: 3.0829 - val_accuracy: 0.5263
Epoch 948/1000
177/177 [==============================] - 0s 94us/step - loss: 0.0064 - accuracy: 1.0000 - val_loss: 3.0861 - val_accuracy: 0.5263
Epoch 949/1000
177/177 [==============================] - 0s 80us/step - loss: 0.0063 - accuracy: 1.0000 - val_loss: 3.0892 - val_accuracy: 0.5263
Epoch 950/1000
177/177 [==============================] - 0s 112us/step - loss: 0.0065 - accuracy: 1.0000 - val_loss: 3.0942 - val_accuracy: 0.5132
Epoch 951/1000
177/177 [==============================] - 0s 133us/step - loss: 0.0064 - accuracy: 1.0000 - val_loss: 3.0896 - val_accuracy: 0.5263
Epoch 952/1000
177/177 [==============================] - 0s 84us/step - loss: 0.0063 - accuracy: 1.0000 - val_loss:

In [216]:
print('train accuracy: %.2f%%' % (np.mean(hist_sel.history['accuracy'])*100))

train accuracy: 99.59%


In [74]:
df_proba5 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_lasso_2.xlsx",
                        sheet_name=0,
                        index_col=None)

In [75]:
df_proba5

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,NY360,2,1,3.848032e-03,0.996096,0.000056
1,p0017kpresabs_qual,NRS113,0,0,8.441349e-01,0.000015,0.155850
2,p0017kpresabs_qual,120337,2,2,1.466081e-02,0.005713,0.979626
3,p0017kpresabs_qual,CFBREBSa127,1,0,9.985392e-01,0.000350,0.001111
4,p0017kpresabs_qual,GA27,2,2,1.665667e-08,0.000005,0.999995
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,CFBREBSa131,2,1,6.617341e-03,0.565843,0.427540
604,p0040presabsSTCC_qual,NRS112,2,2,1.055758e-03,0.045611,0.953333
605,p0040presabsSTCC_qual,BCH-SA-06,0,0,8.747076e-01,0.071162,0.054130
606,p0040presabsSTCC_qual,834N,2,2,2.593112e-06,0.000013,0.999985


In [76]:
y_prob5 = df_proba5[df_proba5['phage']=='p0040presabsSTCC_qual'].iloc[:,-3:]
y_prob5 = y_prob5.to_numpy()
y_prob5

array([[7.51718800e-04, 2.74742440e-03, 9.96500850e-01],
       [8.97230000e-01, 6.08902950e-02, 4.18796280e-02],
       [7.03720650e-05, 4.70860500e-02, 9.52843550e-01],
       [2.88527600e-01, 5.36827740e-01, 1.74644660e-01],
       [1.56823560e-02, 5.86308730e-03, 9.78454600e-01],
       [1.56823560e-02, 5.86308730e-03, 9.78454600e-01],
       [1.36831460e-04, 2.30222500e-02, 9.76841000e-01],
       [2.20663850e-04, 3.77053430e-02, 9.62074000e-01],
       [2.82139740e-06, 1.99340170e-05, 9.99977230e-01],
       [2.77909400e-04, 9.99516370e-01, 2.05717790e-04],
       [3.33344770e-04, 9.94445150e-01, 5.22156530e-03],
       [9.08604400e-03, 3.01494360e-02, 9.60764500e-01],
       [5.34160700e-03, 1.60996360e-01, 8.33662100e-01],
       [2.50591040e-01, 6.41792540e-01, 1.07616420e-01],
       [2.28065280e-06, 5.25360700e-01, 4.74636970e-01],
       [2.56176800e-01, 6.57620650e-02, 6.78061100e-01],
       [9.99708600e-01, 2.64059600e-04, 2.72949020e-05],
       [1.34376030e-01, 9.95259

In [77]:
ovo5 = rocauc_ovo(y_sel_test, y_prob5, average="macro", multi_class="ovo")
ovo5

0.6868501362427706

In [78]:
ovr5 = rocauc_ovr(y_sel_test, y_prob5, average="macro", multi_class="ovr")
ovr5

0.6868501362427706

In [79]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train, X_sel_test, y_sel_train, y_sel_test = train_test_split(X_sel, y_sel,
                                                    test_size = 0.3,
                                                    random_state=678,
                                                    stratify=y_sel)

In [80]:
dat6 = pd.DataFrame(X_sel_test.iloc[:,-1])
dat6['test'] = y_sel_test

In [81]:
dat6

,strain,test
152,NRS169,1
210,NRS249,2
218,NRS262,2
238,SR1746,1
113,NRS029,1
...,...,...
96,GA27,2
95,GA231,2
237,SR1287,0
14,506,2


In [82]:
X_sel_train = X_sel_train.drop(['strain'], axis=1)
X_sel_test = X_sel_test.drop(['strain'], axis=1)

In [83]:
model_sel2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train.shape[1],)),
    Dense(3, activation='softmax'),
])

In [95]:
model_sel2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [96]:
model_sel2.fit(X_sel_train, y_sel_train,
          batch_size=16, epochs=1000,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 693us/step - loss: 11.0636 - accuracy: 0.3672 - val_loss: 7.6774 - val_accuracy: 0.4211
Epoch 2/1000
177/177 [==============================] - 0s 145us/step - loss: 6.2336 - accuracy: 0.4011 - val_loss: 3.9471 - val_accuracy: 0.3684
Epoch 3/1000
177/177 [==============================] - 0s 195us/step - loss: 2.7264 - accuracy: 0.3785 - val_loss: 1.9062 - val_accuracy: 0.3026
Epoch 4/1000
177/177 [==============================] - 0s 171us/step - loss: 1.3789 - accuracy: 0.4689 - val_loss: 1.1360 - val_accuracy: 0.4474
Epoch 5/1000
177/177 [==============================] - 0s 197us/step - loss: 1.2389 - accuracy: 0.5198 - val_loss: 1.0760 - val_accuracy: 0.4737
Epoch 6/1000
177/177 [==============================] - 0s 157us/step - loss: 1.2594 - accuracy: 0.4802 - val_loss: 1.1577 - val_accuracy: 0.4474
Epoch 7/1000
177/177 [==============================] - 0s 166us/step - loss: 

177/177 [==============================] - 0s 119us/step - loss: 0.4583 - accuracy: 0.8927 - val_loss: 1.2551 - val_accuracy: 0.5263
Epoch 112/1000
177/177 [==============================] - 0s 135us/step - loss: 0.4491 - accuracy: 0.8983 - val_loss: 1.2316 - val_accuracy: 0.5132
Epoch 113/1000
177/177 [==============================] - 0s 112us/step - loss: 0.3863 - accuracy: 0.8983 - val_loss: 1.2906 - val_accuracy: 0.5132
Epoch 114/1000
177/177 [==============================] - 0s 118us/step - loss: 0.3729 - accuracy: 0.9040 - val_loss: 1.2399 - val_accuracy: 0.5526
Epoch 115/1000
177/177 [==============================] - 0s 120us/step - loss: 0.2952 - accuracy: 0.9209 - val_loss: 1.2554 - val_accuracy: 0.4737
Epoch 116/1000
177/177 [==============================] - 0s 139us/step - loss: 0.4063 - accuracy: 0.8983 - val_loss: 1.3251 - val_accuracy: 0.4737
Epoch 117/1000
177/177 [==============================] - 0s 212us/step - loss: 0.4822 - accuracy: 0.8757 - val_loss: 1.3190 - 

177/177 [==============================] - 0s 128us/step - loss: 0.1967 - accuracy: 0.9435 - val_loss: 1.6162 - val_accuracy: 0.5789
Epoch 222/1000
177/177 [==============================] - 0s 127us/step - loss: 0.2557 - accuracy: 0.9435 - val_loss: 1.6286 - val_accuracy: 0.5658
Epoch 223/1000
177/177 [==============================] - 0s 156us/step - loss: 0.2644 - accuracy: 0.9153 - val_loss: 1.7422 - val_accuracy: 0.5000
Epoch 224/1000
177/177 [==============================] - 0s 149us/step - loss: 0.4041 - accuracy: 0.9266 - val_loss: 1.7942 - val_accuracy: 0.5263
Epoch 225/1000
177/177 [==============================] - 0s 172us/step - loss: 0.4156 - accuracy: 0.9266 - val_loss: 1.6383 - val_accuracy: 0.5658
Epoch 226/1000
177/177 [==============================] - 0s 182us/step - loss: 0.2547 - accuracy: 0.9322 - val_loss: 1.8455 - val_accuracy: 0.5132
Epoch 227/1000
177/177 [==============================] - 0s 139us/step - loss: 0.3220 - accuracy: 0.9322 - val_loss: 1.6368 - 

177/177 [==============================] - 0s 303us/step - loss: 0.2204 - accuracy: 0.9492 - val_loss: 1.7964 - val_accuracy: 0.5526
Epoch 332/1000
177/177 [==============================] - 0s 232us/step - loss: 0.1843 - accuracy: 0.9548 - val_loss: 1.8708 - val_accuracy: 0.5395
Epoch 333/1000
177/177 [==============================] - 0s 366us/step - loss: 0.3672 - accuracy: 0.9096 - val_loss: 1.9051 - val_accuracy: 0.5658
Epoch 334/1000
177/177 [==============================] - 0s 323us/step - loss: 0.1522 - accuracy: 0.9661 - val_loss: 1.9382 - val_accuracy: 0.5263
Epoch 335/1000
177/177 [==============================] - 0s 284us/step - loss: 0.1827 - accuracy: 0.9548 - val_loss: 1.9061 - val_accuracy: 0.5526
Epoch 336/1000
177/177 [==============================] - 0s 184us/step - loss: 0.2201 - accuracy: 0.9379 - val_loss: 1.9701 - val_accuracy: 0.5000
Epoch 337/1000
177/177 [==============================] - 0s 131us/step - loss: 0.1625 - accuracy: 0.9605 - val_loss: 1.9307 - 

177/177 [==============================] - 0s 167us/step - loss: 0.1816 - accuracy: 0.9492 - val_loss: 2.3142 - val_accuracy: 0.5263
Epoch 442/1000
177/177 [==============================] - 0s 149us/step - loss: 0.0970 - accuracy: 0.9605 - val_loss: 2.2613 - val_accuracy: 0.5395
Epoch 443/1000
177/177 [==============================] - 0s 143us/step - loss: 0.2004 - accuracy: 0.9548 - val_loss: 2.3064 - val_accuracy: 0.5263
Epoch 444/1000
177/177 [==============================] - 0s 151us/step - loss: 0.0992 - accuracy: 0.9605 - val_loss: 2.2690 - val_accuracy: 0.5526
Epoch 445/1000
177/177 [==============================] - 0s 157us/step - loss: 0.1551 - accuracy: 0.9548 - val_loss: 2.2855 - val_accuracy: 0.5263
Epoch 446/1000
177/177 [==============================] - 0s 138us/step - loss: 0.2295 - accuracy: 0.9492 - val_loss: 2.2720 - val_accuracy: 0.5395
Epoch 447/1000
177/177 [==============================] - 0s 129us/step - loss: 0.1112 - accuracy: 0.9605 - val_loss: 2.2215 - 

177/177 [==============================] - 0s 166us/step - loss: 0.1107 - accuracy: 0.9661 - val_loss: 2.5432 - val_accuracy: 0.5132
Epoch 552/1000
177/177 [==============================] - 0s 152us/step - loss: 0.0862 - accuracy: 0.9661 - val_loss: 2.5078 - val_accuracy: 0.5395
Epoch 553/1000
177/177 [==============================] - 0s 186us/step - loss: 0.1415 - accuracy: 0.9492 - val_loss: 2.6672 - val_accuracy: 0.5000
Epoch 554/1000
177/177 [==============================] - 0s 171us/step - loss: 0.2555 - accuracy: 0.9492 - val_loss: 2.6552 - val_accuracy: 0.5132
Epoch 555/1000
177/177 [==============================] - 0s 151us/step - loss: 0.2439 - accuracy: 0.9548 - val_loss: 2.5189 - val_accuracy: 0.5395
Epoch 556/1000
177/177 [==============================] - 0s 184us/step - loss: 0.0940 - accuracy: 0.9661 - val_loss: 3.0090 - val_accuracy: 0.5000
Epoch 557/1000
177/177 [==============================] - 0s 163us/step - loss: 0.6340 - accuracy: 0.9322 - val_loss: 2.7732 - 

177/177 [==============================] - 0s 152us/step - loss: 0.1172 - accuracy: 0.9661 - val_loss: 2.7099 - val_accuracy: 0.5526
Epoch 662/1000
177/177 [==============================] - 0s 188us/step - loss: 0.1106 - accuracy: 0.9661 - val_loss: 2.7194 - val_accuracy: 0.5395
Epoch 663/1000
177/177 [==============================] - 0s 147us/step - loss: 0.1121 - accuracy: 0.9548 - val_loss: 2.6969 - val_accuracy: 0.5658
Epoch 664/1000
177/177 [==============================] - 0s 161us/step - loss: 0.1358 - accuracy: 0.9548 - val_loss: 2.7825 - val_accuracy: 0.5263
Epoch 665/1000
177/177 [==============================] - 0s 218us/step - loss: 0.0800 - accuracy: 0.9718 - val_loss: 2.7421 - val_accuracy: 0.5263
Epoch 666/1000
177/177 [==============================] - 0s 174us/step - loss: 0.1220 - accuracy: 0.9661 - val_loss: 2.6409 - val_accuracy: 0.5263
Epoch 667/1000
177/177 [==============================] - 0s 204us/step - loss: 0.0707 - accuracy: 0.9661 - val_loss: 2.5208 - 

177/177 [==============================] - 0s 151us/step - loss: 0.0723 - accuracy: 0.9548 - val_loss: 3.0074 - val_accuracy: 0.5395
Epoch 772/1000
177/177 [==============================] - 0s 155us/step - loss: 0.0903 - accuracy: 0.9605 - val_loss: 3.0203 - val_accuracy: 0.5658
Epoch 773/1000
177/177 [==============================] - 0s 163us/step - loss: 0.0868 - accuracy: 0.9718 - val_loss: 2.9400 - val_accuracy: 0.5658
Epoch 774/1000
177/177 [==============================] - 0s 147us/step - loss: 0.0656 - accuracy: 0.9605 - val_loss: 2.9180 - val_accuracy: 0.5526
Epoch 775/1000
177/177 [==============================] - 0s 183us/step - loss: 0.1094 - accuracy: 0.9605 - val_loss: 2.8933 - val_accuracy: 0.5658
Epoch 776/1000
177/177 [==============================] - 0s 145us/step - loss: 0.0606 - accuracy: 0.9718 - val_loss: 2.9928 - val_accuracy: 0.5658
Epoch 777/1000
177/177 [==============================] - 0s 165us/step - loss: 0.0920 - accuracy: 0.9718 - val_loss: 3.0330 - 

177/177 [==============================] - 0s 154us/step - loss: 0.0686 - accuracy: 0.9661 - val_loss: 3.1948 - val_accuracy: 0.5395
Epoch 882/1000
177/177 [==============================] - 0s 146us/step - loss: 0.1132 - accuracy: 0.9661 - val_loss: 3.2311 - val_accuracy: 0.5132
Epoch 883/1000
177/177 [==============================] - 0s 137us/step - loss: 0.0475 - accuracy: 0.9774 - val_loss: 3.1568 - val_accuracy: 0.5395
Epoch 884/1000
177/177 [==============================] - 0s 180us/step - loss: 0.0766 - accuracy: 0.9605 - val_loss: 3.1689 - val_accuracy: 0.5132
Epoch 885/1000
177/177 [==============================] - 0s 164us/step - loss: 0.1420 - accuracy: 0.9718 - val_loss: 3.2557 - val_accuracy: 0.5000
Epoch 886/1000
177/177 [==============================] - 0s 152us/step - loss: 0.1308 - accuracy: 0.9661 - val_loss: 3.2132 - val_accuracy: 0.5395
Epoch 887/1000
177/177 [==============================] - 0s 203us/step - loss: 0.1231 - accuracy: 0.9605 - val_loss: 3.2445 - 

177/177 [==============================] - 0s 179us/step - loss: 0.0560 - accuracy: 0.9718 - val_loss: 3.3077 - val_accuracy: 0.5132
Epoch 992/1000
177/177 [==============================] - 0s 143us/step - loss: 0.0531 - accuracy: 0.9661 - val_loss: 3.3043 - val_accuracy: 0.5395
Epoch 993/1000
177/177 [==============================] - 0s 151us/step - loss: 0.1384 - accuracy: 0.9605 - val_loss: 3.2311 - val_accuracy: 0.5526
Epoch 994/1000
177/177 [==============================] - 0s 167us/step - loss: 0.0602 - accuracy: 0.9661 - val_loss: 3.2882 - val_accuracy: 0.5395
Epoch 995/1000
177/177 [==============================] - 0s 142us/step - loss: 0.1372 - accuracy: 0.9605 - val_loss: 3.3018 - val_accuracy: 0.5263
Epoch 996/1000
177/177 [==============================] - 0s 143us/step - loss: 0.0731 - accuracy: 0.9661 - val_loss: 3.2685 - val_accuracy: 0.5132
Epoch 997/1000
177/177 [==============================] - 0s 181us/step - loss: 0.1528 - accuracy: 0.9661 - val_loss: 3.3994 - 

In [224]:
acc_test_sel2 = model_sel2.evaluate(X_sel_test, y_sel_test)[1]
print('test accuracy: %.2f%%' % (acc_test_sel2*100))

76/76 [==============================] - 0s 88us/step
test accuracy: 57.89%


In [97]:
pred6 = model_sel2.predict_classes(X_sel_test)
pred6

array([1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 2, 2, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 1, 2, 2, 1, 0, 2, 0, 0, 2, 1, 2, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 2, 1, 2, 1, 2, 1, 1, 1, 0, 2, 1, 0,
       1, 1, 0, 2, 1, 1, 1, 1, 0, 1])

In [98]:
dat6['pred'] = pred6
dat6

,strain,test,pred
152,NRS169,1,1
210,NRS249,2,1
218,NRS262,2,2
238,SR1746,1,1
113,NRS029,1,1
...,...,...,...
96,GA27,2,1
95,GA231,2,1
237,SR1287,0,1
14,506,2,0


In [99]:
proba6 = model_sel2.predict_proba(X_sel_test)
dat_proba6 = pd.DataFrame(proba6)

In [100]:
dat_proba6

,0,1,2
0,3.419365e-09,0.968961,3.103886e-02
1,8.290029e-08,0.999955,4.508791e-05
2,3.233680e-02,0.133059,8.346044e-01
3,1.945794e-10,0.999999,7.537894e-07
4,2.306629e-05,0.999977,3.929847e-08
...,...,...,...
71,3.964591e-03,0.995929,1.068284e-04
72,3.309226e-04,0.999669,6.232397e-10
73,8.445997e-06,0.999992,1.182947e-13
74,6.516150e-01,0.014809,3.335762e-01


In [101]:
dat_proba6.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba6.csv", index = False,
         header=None)

In [102]:
dat6.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/6p40pST.csv", index = False,
         header=None)

In [228]:
hist_sel2 = model_sel2.fit(X_sel_train, y_sel_train,
          batch_size=16, epochs=1000,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 220us/step - loss: 0.0498 - accuracy: 0.9774 - val_loss: 3.3403 - val_accuracy: 0.5658
Epoch 2/1000
177/177 [==============================] - 0s 298us/step - loss: 0.0748 - accuracy: 0.9661 - val_loss: 3.2625 - val_accuracy: 0.5921
Epoch 3/1000
177/177 [==============================] - 0s 237us/step - loss: 0.0803 - accuracy: 0.9661 - val_loss: 3.2721 - val_accuracy: 0.5921
Epoch 4/1000
177/177 [==============================] - 0s 189us/step - loss: 0.1154 - accuracy: 0.9605 - val_loss: 3.3236 - val_accuracy: 0.5658
Epoch 5/1000
177/177 [==============================] - 0s 182us/step - loss: 0.0495 - accuracy: 0.9661 - val_loss: 3.3043 - val_accuracy: 0.5921
Epoch 6/1000
177/177 [==============================] - 0s 175us/step - loss: 0.0489 - accuracy: 0.9661 - val_loss: 3.2807 - val_accuracy: 0.6053
Epoch 7/1000
177/177 [==============================] - 0s 187us/step - loss: 0

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.120453). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


177/177 [==============================] - 0s 347us/step - loss: 0.0478 - accuracy: 0.9774 - val_loss: 3.3345 - val_accuracy: 0.6053
Epoch 111/1000
177/177 [==============================] - 0s 245us/step - loss: 0.0484 - accuracy: 0.9661 - val_loss: 3.3098 - val_accuracy: 0.6053
Epoch 112/1000
177/177 [==============================] - 0s 235us/step - loss: 0.0470 - accuracy: 0.9718 - val_loss: 3.3175 - val_accuracy: 0.5921
Epoch 113/1000
177/177 [==============================] - 0s 248us/step - loss: 0.0464 - accuracy: 0.9774 - val_loss: 3.3320 - val_accuracy: 0.5921
Epoch 114/1000
177/177 [==============================] - 0s 246us/step - loss: 0.0462 - accuracy: 0.9774 - val_loss: 3.3336 - val_accuracy: 0.5789
Epoch 115/1000
177/177 [==============================] - 0s 190us/step - loss: 0.0456 - accuracy: 0.9718 - val_loss: 3.3391 - val_accuracy: 0.5921
Epoch 116/1000
177/177 [==============================] - 0s 206us/step - loss: 0.0470 - accuracy: 0.9774 - val_loss: 3.3331 - 

177/177 [==============================] - 0s 238us/step - loss: 0.0695 - accuracy: 0.9774 - val_loss: 3.8298 - val_accuracy: 0.5526
Epoch 221/1000
177/177 [==============================] - 0s 257us/step - loss: 0.1713 - accuracy: 0.9605 - val_loss: 3.7148 - val_accuracy: 0.5658
Epoch 222/1000
177/177 [==============================] - 0s 496us/step - loss: 0.1295 - accuracy: 0.9548 - val_loss: 3.7496 - val_accuracy: 0.5789
Epoch 223/1000
177/177 [==============================] - 0s 394us/step - loss: 0.0629 - accuracy: 0.9718 - val_loss: 3.6574 - val_accuracy: 0.5921
Epoch 224/1000
177/177 [==============================] - 0s 284us/step - loss: 0.2829 - accuracy: 0.9322 - val_loss: 3.7138 - val_accuracy: 0.5658
Epoch 225/1000
177/177 [==============================] - 0s 202us/step - loss: 0.0770 - accuracy: 0.9774 - val_loss: 3.6706 - val_accuracy: 0.5789
Epoch 226/1000
177/177 [==============================] - 0s 181us/step - loss: 0.0449 - accuracy: 0.9774 - val_loss: 3.6204 - 

177/177 [==============================] - 0s 175us/step - loss: 0.4372 - accuracy: 0.9605 - val_loss: 4.5272 - val_accuracy: 0.5526
Epoch 331/1000
177/177 [==============================] - 0s 234us/step - loss: 0.2533 - accuracy: 0.9661 - val_loss: 3.9415 - val_accuracy: 0.5921
Epoch 332/1000
177/177 [==============================] - 0s 239us/step - loss: 0.0945 - accuracy: 0.9661 - val_loss: 3.7753 - val_accuracy: 0.5789
Epoch 333/1000
177/177 [==============================] - 0s 245us/step - loss: 0.0684 - accuracy: 0.9718 - val_loss: 3.8203 - val_accuracy: 0.5658
Epoch 334/1000
177/177 [==============================] - 0s 275us/step - loss: 0.1179 - accuracy: 0.9605 - val_loss: 3.8042 - val_accuracy: 0.5789
Epoch 335/1000
177/177 [==============================] - 0s 213us/step - loss: 0.0606 - accuracy: 0.9605 - val_loss: 3.8185 - val_accuracy: 0.5526
Epoch 336/1000
177/177 [==============================] - 0s 214us/step - loss: 0.0503 - accuracy: 0.9718 - val_loss: 4.0390 - 

...

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.140514). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


177/177 [==============================] - 0s 634us/step - loss: 0.1169 - accuracy: 0.9774 - val_loss: 4.4344 - val_accuracy: 0.5789
Epoch 602/1000
177/177 [==============================] - 0s 201us/step - loss: 0.1257 - accuracy: 0.9774 - val_loss: 4.5802 - val_accuracy: 0.5789
Epoch 603/1000
177/177 [==============================] - 0s 288us/step - loss: 0.0289 - accuracy: 0.9774 - val_loss: 4.4825 - val_accuracy: 0.5658
Epoch 604/1000
177/177 [==============================] - 0s 285us/step - loss: 0.1082 - accuracy: 0.9774 - val_loss: 4.5922 - val_accuracy: 0.5658
Epoch 605/1000
177/177 [==============================] - 0s 172us/step - loss: 0.1058 - accuracy: 0.9831 - val_loss: 4.4435 - val_accuracy: 0.5658
Epoch 606/1000
177/177 [==============================] - 0s 146us/step - loss: 0.0814 - accuracy: 0.9774 - val_loss: 4.5428 - val_accuracy: 0.5658
Epoch 607/1000
177/177 [==============================] - 0s 152us/step - loss: 0.1558 - accuracy: 0.9718 - val_loss: 4.6099 - 

177/177 [==============================] - 0s 127us/step - loss: 0.1319 - accuracy: 0.9831 - val_loss: 4.6996 - val_accuracy: 0.5395
Epoch 712/1000
177/177 [==============================] - 0s 118us/step - loss: 0.1627 - accuracy: 0.9661 - val_loss: 4.6157 - val_accuracy: 0.5526
Epoch 713/1000
177/177 [==============================] - 0s 122us/step - loss: 0.0448 - accuracy: 0.9831 - val_loss: 4.4909 - val_accuracy: 0.5526
Epoch 714/1000
177/177 [==============================] - 0s 124us/step - loss: 0.0380 - accuracy: 0.9887 - val_loss: 4.4822 - val_accuracy: 0.5658
Epoch 715/1000
177/177 [==============================] - 0s 129us/step - loss: 0.0322 - accuracy: 0.9831 - val_loss: 4.5131 - val_accuracy: 0.5526
Epoch 716/1000
177/177 [==============================] - 0s 120us/step - loss: 0.0292 - accuracy: 0.9831 - val_loss: 4.5208 - val_accuracy: 0.5526
Epoch 717/1000
177/177 [==============================] - 0s 118us/step - loss: 0.0291 - accuracy: 0.9887 - val_loss: 4.5321 - 

177/177 [==============================] - 0s 157us/step - loss: 0.0340 - accuracy: 0.9887 - val_loss: 4.8138 - val_accuracy: 0.5658
Epoch 822/1000
177/177 [==============================] - 0s 175us/step - loss: 0.0351 - accuracy: 0.9887 - val_loss: 4.7501 - val_accuracy: 0.5658
Epoch 823/1000
177/177 [==============================] - 0s 155us/step - loss: 0.0365 - accuracy: 0.9887 - val_loss: 4.7523 - val_accuracy: 0.5658
Epoch 824/1000
177/177 [==============================] - 0s 149us/step - loss: 0.0324 - accuracy: 0.9831 - val_loss: 4.7880 - val_accuracy: 0.5658
Epoch 825/1000
177/177 [==============================] - 0s 155us/step - loss: 0.0411 - accuracy: 0.9774 - val_loss: 4.7306 - val_accuracy: 0.5658
Epoch 826/1000
177/177 [==============================] - 0s 158us/step - loss: 0.0325 - accuracy: 0.9887 - val_loss: 4.8029 - val_accuracy: 0.5658
Epoch 827/1000
177/177 [==============================] - 0s 145us/step - loss: 0.0389 - accuracy: 0.9831 - val_loss: 4.7064 - 

177/177 [==============================] - 0s 149us/step - loss: 0.0273 - accuracy: 0.9774 - val_loss: 4.9022 - val_accuracy: 0.5658
Epoch 932/1000
177/177 [==============================] - 0s 156us/step - loss: 0.0269 - accuracy: 0.9831 - val_loss: 4.9295 - val_accuracy: 0.5526
Epoch 933/1000
177/177 [==============================] - ETA: 0s - loss: 0.0029 - accuracy: 1.00 - 0s 156us/step - loss: 0.0268 - accuracy: 0.9831 - val_loss: 4.9384 - val_accuracy: 0.5526
Epoch 934/1000
177/177 [==============================] - 0s 161us/step - loss: 0.0270 - accuracy: 0.9831 - val_loss: 4.9107 - val_accuracy: 0.5658
Epoch 935/1000
177/177 [==============================] - 0s 156us/step - loss: 0.0267 - accuracy: 0.9831 - val_loss: 4.9300 - val_accuracy: 0.5526
Epoch 936/1000
177/177 [==============================] - 0s 154us/step - loss: 0.0268 - accuracy: 0.9831 - val_loss: 4.9666 - val_accuracy: 0.5526
Epoch 937/1000
177/177 [==============================] - 0s 150us/step - loss: 0.026

In [229]:
print('train accuracy: %.2f%%' % (np.mean(hist_sel2.history['accuracy'])*100))

train accuracy: 96.98%


In [84]:
df_proba6 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_lasso_2.xlsx",
                        sheet_name=1,
                        index_col=None)

In [85]:
df_proba6

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,CFBREBSa104,1,2,1.290039e-09,1.567630e-07,9.999999e-01
1,p0017kpresabs_qual,NRS199,2,2,2.856965e-05,2.843749e-03,9.971277e-01
2,p0017kpresabs_qual,NRS233,1,0,9.999446e-01,4.541289e-06,5.090974e-05
3,p0017kpresabs_qual,SR1746,0,2,6.282367e-02,7.075194e-04,9.364688e-01
4,p0017kpresabs_qual,NRS202,2,2,8.229589e-03,2.163908e-05,9.917488e-01
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,GA27,2,1,3.964591e-03,9.959286e-01,1.068284e-04
604,p0040presabsSTCC_qual,GA231,2,1,3.309226e-04,9.996691e-01,6.232397e-10
605,p0040presabsSTCC_qual,SR1287,0,1,8.445997e-06,9.999915e-01,1.182947e-13
606,p0040presabsSTCC_qual,506,2,0,6.516150e-01,1.480882e-02,3.335762e-01


In [86]:
y_prob6 = df_proba6[df_proba6['phage']=='p0040presabsSTCC_qual'].iloc[:,-3:]
y_prob6 = y_prob6.to_numpy()
y_prob6

array([[3.41936480e-09, 9.68961200e-01, 3.10388560e-02],
       [8.29002900e-08, 9.99954800e-01, 4.50879120e-05],
       [3.23367980e-02, 1.33058830e-01, 8.34604400e-01],
       [1.94579440e-10, 9.99999300e-01, 7.53789400e-07],
       [2.30662860e-05, 9.99977000e-01, 3.92984670e-08],
       [6.76289450e-12, 1.00000000e+00, 5.51817240e-21],
       [1.69047770e-04, 9.34854450e-01, 6.49765000e-02],
       [1.25273660e-06, 8.30723900e-01, 1.69274880e-01],
       [1.48787900e-07, 9.72064440e-04, 9.99027850e-01],
       [3.34367240e-04, 9.99656200e-01, 9.43186750e-06],
       [4.58884960e-04, 1.02441110e-01, 8.97100000e-01],
       [4.00466840e-03, 2.33040730e-03, 9.93665000e-01],
       [7.96433900e-02, 9.20356630e-01, 1.69437230e-10],
       [4.37546670e-01, 5.60030160e-01, 2.42317000e-03],
       [6.14800700e-03, 9.93722500e-01, 1.29520080e-04],
       [7.49776400e-05, 9.99783460e-01, 1.41584650e-04],
       [6.26841700e-12, 3.48335700e-01, 6.51664260e-01],
       [6.13096700e-05, 8.84887

In [87]:
ovo6 = rocauc_ovo(y_sel_test, y_prob6, average="macro", multi_class="ovo")
ovo6

0.6915930436913519

In [88]:
ovr6 = rocauc_ovr(y_sel_test, y_prob6, average="macro", multi_class="ovr")
ovr6

0.6915930436913519

In [89]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train, X_sel_test, y_sel_train, y_sel_test = train_test_split(X_sel, y_sel,
                                                    test_size = 0.3,
                                                    random_state=789,
                                                    stratify=y_sel)

In [90]:
dat7 = pd.DataFrame(X_sel_test.iloc[:,-1])
dat7['test'] = y_sel_test

In [91]:
dat7

,strain,test
120,NRS063,1
187,NRS219,2
96,GA27,2
53,CFBREBSa119,1
197,NRS233,2
...,...,...
47,CFBREBSa110,2
68,CFBRSa05,0
56,CFBREBSa123,0
231,NY360,1


In [92]:
X_sel_train = X_sel_train.drop(['strain'], axis=1)
X_sel_test = X_sel_test.drop(['strain'], axis=1)

In [93]:
model_sel3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train.shape[1],)),
    Dense(3, activation='softmax'),
])

In [108]:
model_sel3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [109]:
model_sel3.fit(X_sel_train, y_sel_train,
          batch_size=64, epochs=1000,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 628us/step - loss: 18.3438 - accuracy: 0.3220 - val_loss: 23.4208 - val_accuracy: 0.3158
Epoch 2/1000
177/177 [==============================] - 0s 63us/step - loss: 16.2084 - accuracy: 0.3333 - val_loss: 21.2330 - val_accuracy: 0.3553
Epoch 3/1000
177/177 [==============================] - 0s 119us/step - loss: 14.1420 - accuracy: 0.3616 - val_loss: 19.1816 - val_accuracy: 0.3947
Epoch 4/1000
177/177 [==============================] - 0s 75us/step - loss: 12.4666 - accuracy: 0.3729 - val_loss: 17.1185 - val_accuracy: 0.3553
Epoch 5/1000
177/177 [==============================] - 0s 102us/step - loss: 10.5170 - accuracy: 0.3390 - val_loss: 15.1804 - val_accuracy: 0.4079
Epoch 6/1000
177/177 [==============================] - 0s 116us/step - loss: 8.9393 - accuracy: 0.3277 - val_loss: 14.0285 - val_accuracy: 0.3026
Epoch 7/1000
177/177 [==============================] - 0s 102us/step 

177/177 [==============================] - 0s 128us/step - loss: 0.4577 - accuracy: 0.8418 - val_loss: 1.2041 - val_accuracy: 0.4868
Epoch 113/1000
177/177 [==============================] - 0s 135us/step - loss: 0.4550 - accuracy: 0.8588 - val_loss: 1.1937 - val_accuracy: 0.5000
Epoch 114/1000
177/177 [==============================] - 0s 77us/step - loss: 0.4420 - accuracy: 0.8644 - val_loss: 1.2214 - val_accuracy: 0.5000
Epoch 115/1000
177/177 [==============================] - 0s 81us/step - loss: 0.4527 - accuracy: 0.8644 - val_loss: 1.2199 - val_accuracy: 0.5000
Epoch 116/1000
177/177 [==============================] - 0s 85us/step - loss: 0.4386 - accuracy: 0.8757 - val_loss: 1.1762 - val_accuracy: 0.5000
Epoch 117/1000
177/177 [==============================] - 0s 87us/step - loss: 0.4486 - accuracy: 0.8531 - val_loss: 1.1669 - val_accuracy: 0.5132
Epoch 118/1000
177/177 [==============================] - 0s 77us/step - loss: 0.4408 - accuracy: 0.8531 - val_loss: 1.2104 - val_a

Epoch 168/1000
177/177 [==============================] - 0s 385us/step - loss: 0.3301 - accuracy: 0.9040 - val_loss: 1.3034 - val_accuracy: 0.5000
Epoch 169/1000
177/177 [==============================] - 0s 443us/step - loss: 0.3277 - accuracy: 0.8983 - val_loss: 1.3379 - val_accuracy: 0.5132
Epoch 170/1000
177/177 [==============================] - 0s 154us/step - loss: 0.3326 - accuracy: 0.9040 - val_loss: 1.3273 - val_accuracy: 0.5132
Epoch 171/1000
177/177 [==============================] - 0s 107us/step - loss: 0.3233 - accuracy: 0.9096 - val_loss: 1.3190 - val_accuracy: 0.5000
Epoch 172/1000
177/177 [==============================] - 0s 79us/step - loss: 0.3221 - accuracy: 0.9153 - val_loss: 1.3352 - val_accuracy: 0.5000
Epoch 173/1000
177/177 [==============================] - 0s 73us/step - loss: 0.3211 - accuracy: 0.9153 - val_loss: 1.3086 - val_accuracy: 0.4868
Epoch 174/1000
177/177 [==============================] - 0s 63us/step - loss: 0.3165 - accuracy: 0.9153 - val_los

Epoch 279/1000
177/177 [==============================] - 0s 59us/step - loss: 0.2009 - accuracy: 0.9548 - val_loss: 1.6249 - val_accuracy: 0.5395
Epoch 280/1000
177/177 [==============================] - 0s 50us/step - loss: 0.1949 - accuracy: 0.9605 - val_loss: 1.5999 - val_accuracy: 0.5395
Epoch 281/1000
177/177 [==============================] - 0s 53us/step - loss: 0.1985 - accuracy: 0.9379 - val_loss: 1.6116 - val_accuracy: 0.5395
Epoch 282/1000
177/177 [==============================] - 0s 57us/step - loss: 0.2075 - accuracy: 0.9435 - val_loss: 1.6169 - val_accuracy: 0.5395
Epoch 283/1000
177/177 [==============================] - 0s 49us/step - loss: 0.2024 - accuracy: 0.9379 - val_loss: 1.6540 - val_accuracy: 0.5395
Epoch 284/1000
177/177 [==============================] - 0s 55us/step - loss: 0.2427 - accuracy: 0.9266 - val_loss: 1.6179 - val_accuracy: 0.5395
Epoch 285/1000
177/177 [==============================] - 0s 61us/step - loss: 0.1967 - accuracy: 0.9548 - val_loss: 1

Epoch 335/1000
177/177 [==============================] - 0s 53us/step - loss: 0.1613 - accuracy: 0.9718 - val_loss: 1.7715 - val_accuracy: 0.5395
Epoch 336/1000
177/177 [==============================] - 0s 53us/step - loss: 0.1596 - accuracy: 0.9718 - val_loss: 1.7357 - val_accuracy: 0.5395
Epoch 337/1000
177/177 [==============================] - 0s 71us/step - loss: 0.1600 - accuracy: 0.9661 - val_loss: 1.7391 - val_accuracy: 0.5395
Epoch 338/1000
177/177 [==============================] - 0s 64us/step - loss: 0.1580 - accuracy: 0.9661 - val_loss: 1.7749 - val_accuracy: 0.5395
Epoch 339/1000
177/177 [==============================] - 0s 60us/step - loss: 0.1607 - accuracy: 0.9661 - val_loss: 1.7957 - val_accuracy: 0.5395
Epoch 340/1000
177/177 [==============================] - 0s 58us/step - loss: 0.1586 - accuracy: 0.9605 - val_loss: 1.7919 - val_accuracy: 0.5395
Epoch 341/1000
177/177 [==============================] - 0s 73us/step - loss: 0.1682 - accuracy: 0.9548 - val_loss: 1

...

Epoch 667/1000
177/177 [==============================] - 0s 50us/step - loss: 0.0794 - accuracy: 0.9718 - val_loss: 2.5189 - val_accuracy: 0.5395
Epoch 668/1000
177/177 [==============================] - 0s 53us/step - loss: 0.0742 - accuracy: 0.9774 - val_loss: 2.6036 - val_accuracy: 0.5395
Epoch 669/1000
177/177 [==============================] - 0s 50us/step - loss: 0.0909 - accuracy: 0.9718 - val_loss: 2.5252 - val_accuracy: 0.5395
Epoch 670/1000
177/177 [==============================] - 0s 54us/step - loss: 0.0943 - accuracy: 0.9605 - val_loss: 2.4972 - val_accuracy: 0.5526
Epoch 671/1000
177/177 [==============================] - 0s 57us/step - loss: 0.0730 - accuracy: 0.9774 - val_loss: 2.6004 - val_accuracy: 0.5395
Epoch 672/1000
177/177 [==============================] - 0s 52us/step - loss: 0.1299 - accuracy: 0.9661 - val_loss: 2.5091 - val_accuracy: 0.5526
Epoch 673/1000
177/177 [==============================] - 0s 61us/step - loss: 0.1323 - accuracy: 0.9492 - val_loss: 2

Epoch 723/1000
177/177 [==============================] - 0s 76us/step - loss: 0.0665 - accuracy: 0.9774 - val_loss: 2.6117 - val_accuracy: 0.5526
Epoch 724/1000
177/177 [==============================] - 0s 74us/step - loss: 0.0668 - accuracy: 0.9774 - val_loss: 2.6072 - val_accuracy: 0.5526
Epoch 725/1000
177/177 [==============================] - 0s 88us/step - loss: 0.0676 - accuracy: 0.9774 - val_loss: 2.6052 - val_accuracy: 0.5526
Epoch 726/1000
177/177 [==============================] - 0s 64us/step - loss: 0.0656 - accuracy: 0.9774 - val_loss: 2.6437 - val_accuracy: 0.5395
Epoch 727/1000
177/177 [==============================] - 0s 56us/step - loss: 0.0672 - accuracy: 0.9774 - val_loss: 2.6336 - val_accuracy: 0.5395
Epoch 728/1000
177/177 [==============================] - 0s 68us/step - loss: 0.0666 - accuracy: 0.9774 - val_loss: 2.6155 - val_accuracy: 0.5395
Epoch 729/1000
177/177 [==============================] - 0s 74us/step - loss: 0.0664 - accuracy: 0.9774 - val_loss: 2

Epoch 779/1000
177/177 [==============================] - 0s 51us/step - loss: 0.0619 - accuracy: 0.9774 - val_loss: 2.6935 - val_accuracy: 0.5526
Epoch 780/1000
177/177 [==============================] - 0s 58us/step - loss: 0.0610 - accuracy: 0.9774 - val_loss: 2.7086 - val_accuracy: 0.5526
Epoch 781/1000
177/177 [==============================] - 0s 59us/step - loss: 0.0615 - accuracy: 0.9774 - val_loss: 2.7109 - val_accuracy: 0.5526
Epoch 782/1000
177/177 [==============================] - 0s 62us/step - loss: 0.0605 - accuracy: 0.9774 - val_loss: 2.7176 - val_accuracy: 0.5526
Epoch 783/1000
177/177 [==============================] - 0s 68us/step - loss: 0.0611 - accuracy: 0.9774 - val_loss: 2.7215 - val_accuracy: 0.5526
Epoch 784/1000
177/177 [==============================] - 0s 71us/step - loss: 0.0607 - accuracy: 0.9718 - val_loss: 2.7287 - val_accuracy: 0.5395
Epoch 785/1000
177/177 [==============================] - 0s 74us/step - loss: 0.0660 - accuracy: 0.9718 - val_loss: 2

Epoch 835/1000
177/177 [==============================] - 0s 51us/step - loss: 0.0584 - accuracy: 0.9774 - val_loss: 2.7957 - val_accuracy: 0.5526
Epoch 836/1000
177/177 [==============================] - 0s 46us/step - loss: 0.0571 - accuracy: 0.9774 - val_loss: 2.7826 - val_accuracy: 0.5526
Epoch 837/1000
177/177 [==============================] - 0s 51us/step - loss: 0.0586 - accuracy: 0.9774 - val_loss: 2.7776 - val_accuracy: 0.5526
Epoch 838/1000
177/177 [==============================] - 0s 59us/step - loss: 0.0581 - accuracy: 0.9774 - val_loss: 2.8032 - val_accuracy: 0.5526
Epoch 839/1000
177/177 [==============================] - 0s 47us/step - loss: 0.0578 - accuracy: 0.9774 - val_loss: 2.8351 - val_accuracy: 0.5395
Epoch 840/1000
177/177 [==============================] - 0s 46us/step - loss: 0.0597 - accuracy: 0.9774 - val_loss: 2.7988 - val_accuracy: 0.5526
Epoch 841/1000
177/177 [==============================] - 0s 57us/step - loss: 0.0579 - accuracy: 0.9831 - val_loss: 2

Epoch 891/1000
177/177 [==============================] - 0s 80us/step - loss: 0.0681 - accuracy: 0.9661 - val_loss: 3.0103 - val_accuracy: 0.5395
Epoch 892/1000
177/177 [==============================] - 0s 71us/step - loss: 0.1194 - accuracy: 0.9661 - val_loss: 2.8470 - val_accuracy: 0.5526
Epoch 893/1000
177/177 [==============================] - 0s 74us/step - loss: 0.0667 - accuracy: 0.9661 - val_loss: 2.8499 - val_accuracy: 0.5395
Epoch 894/1000
177/177 [==============================] - 0s 81us/step - loss: 0.0557 - accuracy: 0.9718 - val_loss: 2.8941 - val_accuracy: 0.5263
Epoch 895/1000
177/177 [==============================] - 0s 110us/step - loss: 0.0600 - accuracy: 0.9718 - val_loss: 2.8901 - val_accuracy: 0.5395
Epoch 896/1000
177/177 [==============================] - 0s 80us/step - loss: 0.0553 - accuracy: 0.9718 - val_loss: 2.8290 - val_accuracy: 0.5526
Epoch 897/1000
177/177 [==============================] - 0s 105us/step - loss: 0.0588 - accuracy: 0.9774 - val_loss:

In [245]:
acc_test_sel3 = model_sel3.evaluate(X_sel_test, y_sel_test)[1]
print('test accuracy: %.2f%%' % (acc_test_sel3*100))

76/76 [==============================] - 0s 64us/step
test accuracy: 53.95%


In [110]:
pred7 = model_sel3.predict_classes(X_sel_test)
pred7

array([1, 2, 1, 2, 2, 0, 1, 2, 2, 2, 0, 2, 2, 1, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 1, 2, 1, 1, 2, 1, 2, 1, 1, 0, 1, 0, 2, 0, 2, 2, 1,
       0, 0, 1, 1, 1, 2, 2, 2, 1, 1, 0, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2,
       1, 1, 0, 1, 0, 2, 2, 0, 1, 1])

In [111]:
dat7['pred'] = pred7
dat7

,strain,test,pred
120,NRS063,1,1
187,NRS219,2,2
96,GA27,2,1
53,CFBREBSa119,1,2
197,NRS233,2,2
...,...,...,...
47,CFBREBSa110,2,2
68,CFBRSa05,0,2
56,CFBREBSa123,0,0
231,NY360,1,1


In [112]:
proba7 = model_sel3.predict_proba(X_sel_test)
dat_proba7 = pd.DataFrame(proba7)

In [113]:
dat_proba7

,0,1,2
0,0.001667,0.997123,0.001210
1,0.000003,0.000068,0.999929
2,0.209421,0.790520,0.000060
3,0.000002,0.001453,0.998545
4,0.412327,0.004981,0.582692
...,...,...,...
71,0.044135,0.010244,0.945621
72,0.145032,0.408530,0.446438
73,0.994623,0.004152,0.001224
74,0.000795,0.948566,0.050639


In [114]:
dat_proba7.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba7.csv", index = False,
         header=None)

In [115]:
dat7.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/7p40pST.csv", index = False,
         header=None)

In [249]:
hist_sel3 = model_sel3.fit(X_sel_train, y_sel_train,
          batch_size=64, epochs=1000,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 77us/step - loss: 0.0491 - accuracy: 0.9774 - val_loss: 2.9670 - val_accuracy: 0.5395
Epoch 2/1000
177/177 [==============================] - 0s 70us/step - loss: 0.0491 - accuracy: 0.9831 - val_loss: 2.9832 - val_accuracy: 0.5395
Epoch 3/1000
177/177 [==============================] - 0s 58us/step - loss: 0.0496 - accuracy: 0.9831 - val_loss: 2.9822 - val_accuracy: 0.5395
Epoch 4/1000
177/177 [==============================] - 0s 72us/step - loss: 0.0495 - accuracy: 0.9831 - val_loss: 2.9811 - val_accuracy: 0.5395
Epoch 5/1000
177/177 [==============================] - 0s 107us/step - loss: 0.0489 - accuracy: 0.9831 - val_loss: 2.9873 - val_accuracy: 0.5395
Epoch 6/1000
177/177 [==============================] - 0s 83us/step - loss: 0.0493 - accuracy: 0.9831 - val_loss: 2.9726 - val_accuracy: 0.5395
Epoch 7/1000
177/177 [==============================] - 0s 114us/step - loss: 0.0492

Epoch 57/1000
177/177 [==============================] - 0s 56us/step - loss: 0.1064 - accuracy: 0.9605 - val_loss: 2.9470 - val_accuracy: 0.5395
Epoch 58/1000
177/177 [==============================] - 0s 97us/step - loss: 0.1253 - accuracy: 0.9548 - val_loss: 3.0617 - val_accuracy: 0.5000
Epoch 59/1000
177/177 [==============================] - 0s 54us/step - loss: 0.1477 - accuracy: 0.9548 - val_loss: 2.9082 - val_accuracy: 0.5395
Epoch 60/1000
177/177 [==============================] - 0s 70us/step - loss: 0.0927 - accuracy: 0.9718 - val_loss: 2.9422 - val_accuracy: 0.5263
Epoch 61/1000
177/177 [==============================] - 0s 76us/step - loss: 0.0618 - accuracy: 0.9718 - val_loss: 2.8213 - val_accuracy: 0.5395
Epoch 62/1000
177/177 [==============================] - 0s 76us/step - loss: 0.0782 - accuracy: 0.9774 - val_loss: 2.9084 - val_accuracy: 0.5395
Epoch 63/1000
177/177 [==============================] - 0s 75us/step - loss: 0.0518 - accuracy: 0.9774 - val_loss: 3.0510 -

Epoch 113/1000
177/177 [==============================] - 0s 65us/step - loss: 0.0445 - accuracy: 0.9831 - val_loss: 3.1149 - val_accuracy: 0.5395
Epoch 114/1000
177/177 [==============================] - 0s 61us/step - loss: 0.0446 - accuracy: 0.9831 - val_loss: 3.1067 - val_accuracy: 0.5263
Epoch 115/1000
177/177 [==============================] - 0s 88us/step - loss: 0.0445 - accuracy: 0.9831 - val_loss: 3.0999 - val_accuracy: 0.5263
Epoch 116/1000
177/177 [==============================] - 0s 96us/step - loss: 0.0457 - accuracy: 0.9831 - val_loss: 3.1132 - val_accuracy: 0.5263
Epoch 117/1000
177/177 [==============================] - 0s 84us/step - loss: 0.0445 - accuracy: 0.9831 - val_loss: 3.1085 - val_accuracy: 0.5263
Epoch 118/1000
177/177 [==============================] - 0s 81us/step - loss: 0.0445 - accuracy: 0.9831 - val_loss: 3.1144 - val_accuracy: 0.5132
Epoch 119/1000
177/177 [==============================] - 0s 75us/step - loss: 0.0444 - accuracy: 0.9831 - val_loss: 3

Epoch 169/1000
177/177 [==============================] - 0s 180us/step - loss: 0.0433 - accuracy: 0.9831 - val_loss: 3.1766 - val_accuracy: 0.5263
Epoch 170/1000
177/177 [==============================] - 0s 119us/step - loss: 0.0425 - accuracy: 0.9831 - val_loss: 3.1882 - val_accuracy: 0.5263
Epoch 171/1000
177/177 [==============================] - 0s 118us/step - loss: 0.0426 - accuracy: 0.9831 - val_loss: 3.1697 - val_accuracy: 0.5263
Epoch 172/1000
177/177 [==============================] - 0s 394us/step - loss: 0.0425 - accuracy: 0.9831 - val_loss: 3.1530 - val_accuracy: 0.5132
Epoch 173/1000
177/177 [==============================] - 0s 425us/step - loss: 0.0423 - accuracy: 0.9774 - val_loss: 3.1692 - val_accuracy: 0.5132
Epoch 174/1000
177/177 [==============================] - 0s 82us/step - loss: 0.0423 - accuracy: 0.9831 - val_loss: 3.1977 - val_accuracy: 0.5132
Epoch 175/1000
177/177 [==============================] - 0s 72us/step - loss: 0.0421 - accuracy: 0.9831 - val_lo

Epoch 280/1000
177/177 [==============================] - 0s 69us/step - loss: 0.1051 - accuracy: 0.9605 - val_loss: 3.2020 - val_accuracy: 0.5263
Epoch 281/1000
177/177 [==============================] - 0s 144us/step - loss: 0.0425 - accuracy: 0.9831 - val_loss: 3.4626 - val_accuracy: 0.5132
Epoch 282/1000
177/177 [==============================] - 0s 93us/step - loss: 0.1638 - accuracy: 0.9718 - val_loss: 3.2858 - val_accuracy: 0.5132
Epoch 283/1000
177/177 [==============================] - 0s 74us/step - loss: 0.0857 - accuracy: 0.9661 - val_loss: 3.2233 - val_accuracy: 0.5263
Epoch 284/1000
177/177 [==============================] - 0s 75us/step - loss: 0.0626 - accuracy: 0.9718 - val_loss: 3.5001 - val_accuracy: 0.5132
Epoch 285/1000
177/177 [==============================] - 0s 60us/step - loss: 0.2644 - accuracy: 0.9718 - val_loss: 3.3905 - val_accuracy: 0.5132
Epoch 286/1000
177/177 [==============================] - 0s 81us/step - loss: 0.0398 - accuracy: 0.9831 - val_loss: 

...

Epoch 613/1000
177/177 [==============================] - 0s 65us/step - loss: 0.0328 - accuracy: 0.9887 - val_loss: 3.7964 - val_accuracy: 0.5000
Epoch 614/1000
177/177 [==============================] - 0s 72us/step - loss: 0.0319 - accuracy: 0.9887 - val_loss: 3.7882 - val_accuracy: 0.5000
Epoch 615/1000
177/177 [==============================] - 0s 80us/step - loss: 0.0321 - accuracy: 0.9887 - val_loss: 3.7780 - val_accuracy: 0.5132
Epoch 616/1000
177/177 [==============================] - 0s 78us/step - loss: 0.0320 - accuracy: 0.9887 - val_loss: 3.7601 - val_accuracy: 0.5132
Epoch 617/1000
177/177 [==============================] - 0s 78us/step - loss: 0.0323 - accuracy: 0.9887 - val_loss: 3.7529 - val_accuracy: 0.5000
Epoch 618/1000
177/177 [==============================] - 0s 74us/step - loss: 0.0318 - accuracy: 0.9887 - val_loss: 3.7588 - val_accuracy: 0.5000
Epoch 619/1000
177/177 [==============================] - 0s 64us/step - loss: 0.0314 - accuracy: 0.9887 - val_loss: 3

177/177 [==============================] - 0s 77us/step - loss: 0.0317 - accuracy: 0.9887 - val_loss: 3.8484 - val_accuracy: 0.5000
Epoch 724/1000
177/177 [==============================] - 0s 96us/step - loss: 0.0303 - accuracy: 0.9887 - val_loss: 3.8826 - val_accuracy: 0.5132
Epoch 725/1000
177/177 [==============================] - 0s 78us/step - loss: 0.0306 - accuracy: 0.9831 - val_loss: 3.9140 - val_accuracy: 0.5000
Epoch 726/1000
177/177 [==============================] - 0s 91us/step - loss: 0.0361 - accuracy: 0.9774 - val_loss: 3.8274 - val_accuracy: 0.5132
Epoch 727/1000
177/177 [==============================] - 0s 84us/step - loss: 0.0741 - accuracy: 0.9774 - val_loss: 3.8109 - val_accuracy: 0.5000
Epoch 728/1000
177/177 [==============================] - 0s 74us/step - loss: 0.0308 - accuracy: 0.9887 - val_loss: 3.9447 - val_accuracy: 0.5263
Epoch 729/1000
177/177 [==============================] - 0s 123us/step - loss: 0.0622 - accuracy: 0.9774 - val_loss: 3.7550 - val_ac

Epoch 779/1000
177/177 [==============================] - 0s 70us/step - loss: 0.0289 - accuracy: 0.9887 - val_loss: 3.9623 - val_accuracy: 0.5000
Epoch 780/1000
177/177 [==============================] - 0s 72us/step - loss: 0.0292 - accuracy: 0.9887 - val_loss: 3.9731 - val_accuracy: 0.5000
Epoch 781/1000
177/177 [==============================] - 0s 84us/step - loss: 0.0294 - accuracy: 0.9887 - val_loss: 3.9936 - val_accuracy: 0.5000
Epoch 782/1000
177/177 [==============================] - 0s 140us/step - loss: 0.0293 - accuracy: 0.9887 - val_loss: 4.0254 - val_accuracy: 0.5000
Epoch 783/1000
177/177 [==============================] - 0s 113us/step - loss: 0.0293 - accuracy: 0.9887 - val_loss: 4.0228 - val_accuracy: 0.5000
Epoch 784/1000
177/177 [==============================] - 0s 97us/step - loss: 0.0289 - accuracy: 0.9887 - val_loss: 4.0168 - val_accuracy: 0.5000
Epoch 785/1000
177/177 [==============================] - 0s 79us/step - loss: 0.0290 - accuracy: 0.9887 - val_loss:

177/177 [==============================] - 0s 142us/step - loss: 0.0276 - accuracy: 0.9887 - val_loss: 4.1553 - val_accuracy: 0.4868
Epoch 890/1000
177/177 [==============================] - 0s 100us/step - loss: 0.0274 - accuracy: 0.9887 - val_loss: 4.1906 - val_accuracy: 0.4868
Epoch 891/1000
177/177 [==============================] - 0s 135us/step - loss: 0.0281 - accuracy: 0.9887 - val_loss: 4.2048 - val_accuracy: 0.4868
Epoch 892/1000
177/177 [==============================] - 0s 85us/step - loss: 0.0286 - accuracy: 0.9831 - val_loss: 4.1749 - val_accuracy: 0.5000
Epoch 893/1000
177/177 [==============================] - 0s 80us/step - loss: 0.0278 - accuracy: 0.9831 - val_loss: 4.1481 - val_accuracy: 0.5000
Epoch 894/1000
177/177 [==============================] - 0s 75us/step - loss: 0.0273 - accuracy: 0.9887 - val_loss: 4.1521 - val_accuracy: 0.4868
Epoch 895/1000
177/177 [==============================] - 0s 104us/step - loss: 0.0275 - accuracy: 0.9887 - val_loss: 4.1469 - val

177/177 [==============================] - 0s 85us/step - loss: 0.0265 - accuracy: 0.9831 - val_loss: 4.3014 - val_accuracy: 0.4868
Epoch 1000/1000
177/177 [==============================] - 0s 120us/step - loss: 0.0275 - accuracy: 0.9887 - val_loss: 4.3074 - val_accuracy: 0.4868


In [250]:
print('train accuracy: %.2f%%' % (np.mean(hist_sel3.history['accuracy'])*100))

train accuracy: 98.51%


In [94]:
df_proba7 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_lasso_2.xlsx",
                        sheet_name=2,
                        index_col=None)

In [95]:
df_proba7

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,NRS271,1,0,0.854929,0.144940,0.000130
1,p0017kpresabs_qual,SR2852,2,2,0.000001,0.000075,0.999923
2,p0017kpresabs_qual,CA39,2,0,0.959992,0.013406,0.026602
3,p0017kpresabs_qual,NRS266,1,2,0.000018,0.000003,0.999980
4,p0017kpresabs_qual,NY356,2,2,0.009156,0.411991,0.578852
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,CFBREBSa110,2,2,0.044135,0.010244,0.945621
604,p0040presabsSTCC_qual,CFBRSa05,0,2,0.145032,0.408530,0.446439
605,p0040presabsSTCC_qual,CFBREBSa123,0,0,0.994623,0.004152,0.001224
606,p0040presabsSTCC_qual,NY360,1,1,0.000795,0.948566,0.050639


In [96]:
y_prob7 = df_proba7[df_proba7['phage']=='p0040presabsSTCC_qual'].iloc[:,-3:]
y_prob7 = y_prob7.to_numpy()
y_prob7

array([[1.66690190e-03, 9.97123200e-01, 1.20998790e-03],
       [3.49782390e-06, 6.78205000e-05, 9.99928700e-01],
       [2.09420860e-01, 7.90519650e-01, 5.95099200e-05],
       [1.96571390e-06, 1.45322250e-03, 9.98544800e-01],
       [4.12326540e-01, 4.98116340e-03, 5.82692270e-01],
       [7.72617500e-01, 4.32880400e-02, 1.84094460e-01],
       [1.71254470e-01, 4.99478640e-01, 3.29266900e-01],
       [3.80232800e-03, 3.76772940e-01, 6.19424760e-01],
       [2.21669350e-01, 1.51685240e-01, 6.26645450e-01],
       [4.41352540e-02, 1.02438750e-02, 9.45620830e-01],
       [9.92457750e-01, 7.54223900e-03, 2.72427330e-09],
       [9.40488530e-04, 6.44437250e-13, 9.99059500e-01],
       [1.02810030e-09, 1.55061130e-05, 9.99984500e-01],
       [5.64978700e-07, 9.91509100e-01, 8.49027800e-03],
       [8.51652300e-06, 9.92857340e-01, 7.13409670e-03],
       [3.38913600e-06, 9.99820650e-01, 1.75997020e-04],
       [9.99552400e-01, 4.43265980e-04, 4.26081500e-06],
       [8.34376300e-01, 1.75572

In [97]:
ovo7 = rocauc_ovo(y_sel_test, y_prob7, average="macro", multi_class="ovo")
ovo7

0.704575980285179

In [98]:
ovr7 = rocauc_ovr(y_sel_test, y_prob7, average="macro", multi_class="ovr")
ovr7

0.704575980285179

In [99]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train, X_sel_test, y_sel_train, y_sel_test = train_test_split(X_sel, y_sel,
                                                    test_size = 0.3,
                                                    random_state=890,
                                                    stratify=y_sel)

In [100]:
dat8 = pd.DataFrame(X_sel_test.iloc[:,-1])
dat8['test'] = y_sel_test

In [101]:
dat8

,strain,test
130,NRS103,2
92,EUH25,0
143,NRS148,2
107,NRS001,1
166,NRS192,0
...,...,...
139,NRS113,1
24,BCH-SA-09,2
133,NRS106,2
62,CFBREBSa131,2


In [102]:
X_sel_train = X_sel_train.drop(['strain'], axis=1)
X_sel_test = X_sel_test.drop(['strain'], axis=1)

In [103]:
model_sel4 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train.shape[1],)),
    Dense(3, activation='softmax'),
])

In [121]:
model_sel4.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [122]:
model_sel4.fit(X_sel_train, y_sel_train,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 708us/step - loss: 20.2238 - accuracy: 0.2712 - val_loss: 15.0039 - val_accuracy: 0.2105
Epoch 2/1000
177/177 [==============================] - 0s 100us/step - loss: 13.7926 - accuracy: 0.3051 - val_loss: 10.3115 - val_accuracy: 0.1842
Epoch 3/1000
177/177 [==============================] - 0s 156us/step - loss: 8.7917 - accuracy: 0.3107 - val_loss: 6.2796 - val_accuracy: 0.3026
Epoch 4/1000
177/177 [==============================] - 0s 126us/step - loss: 5.4862 - accuracy: 0.4181 - val_loss: 5.0898 - val_accuracy: 0.4079
Epoch 5/1000
177/177 [==============================] - 0s 214us/step - loss: 5.1095 - accuracy: 0.4576 - val_loss: 3.2619 - val_accuracy: 0.3816
Epoch 6/1000
177/177 [==============================] - 0s 176us/step - loss: 2.9017 - accuracy: 0.4407 - val_loss: 1.8542 - val_accuracy: 0.3684
Epoch 7/1000
177/177 [==============================] - 0s 158us/step - los

177/177 [==============================] - 0s 140us/step - loss: 0.4236 - accuracy: 0.8983 - val_loss: 1.0144 - val_accuracy: 0.5000
Epoch 112/1000
177/177 [==============================] - 0s 88us/step - loss: 0.5159 - accuracy: 0.8814 - val_loss: 1.0408 - val_accuracy: 0.5395
Epoch 113/1000
177/177 [==============================] - 0s 125us/step - loss: 0.5242 - accuracy: 0.8814 - val_loss: 1.0234 - val_accuracy: 0.5132
Epoch 114/1000
177/177 [==============================] - 0s 130us/step - loss: 0.3964 - accuracy: 0.8983 - val_loss: 1.0202 - val_accuracy: 0.5000
Epoch 115/1000
177/177 [==============================] - 0s 157us/step - loss: 0.3627 - accuracy: 0.9040 - val_loss: 1.0655 - val_accuracy: 0.5395
Epoch 116/1000
177/177 [==============================] - 0s 139us/step - loss: 0.4097 - accuracy: 0.9096 - val_loss: 1.0737 - val_accuracy: 0.5132
Epoch 117/1000
177/177 [==============================] - 0s 227us/step - loss: 0.3589 - accuracy: 0.9040 - val_loss: 1.0830 - v

177/177 [==============================] - 0s 88us/step - loss: 0.2415 - accuracy: 0.9605 - val_loss: 1.2845 - val_accuracy: 0.5526
Epoch 222/1000
177/177 [==============================] - 0s 80us/step - loss: 0.4238 - accuracy: 0.9435 - val_loss: 1.3582 - val_accuracy: 0.5263
Epoch 223/1000
177/177 [==============================] - 0s 80us/step - loss: 0.4267 - accuracy: 0.9322 - val_loss: 1.4924 - val_accuracy: 0.5000
Epoch 224/1000
177/177 [==============================] - 0s 107us/step - loss: 0.2647 - accuracy: 0.9266 - val_loss: 1.2915 - val_accuracy: 0.5395
Epoch 225/1000
177/177 [==============================] - 0s 115us/step - loss: 0.2102 - accuracy: 0.9435 - val_loss: 1.3150 - val_accuracy: 0.5395
Epoch 226/1000
177/177 [==============================] - 0s 88us/step - loss: 0.1825 - accuracy: 0.9492 - val_loss: 1.2966 - val_accuracy: 0.5132
Epoch 227/1000
177/177 [==============================] - 0s 80us/step - loss: 0.2354 - accuracy: 0.9492 - val_loss: 1.3194 - val_a

177/177 [==============================] - 0s 87us/step - loss: 0.1798 - accuracy: 0.9718 - val_loss: 1.5670 - val_accuracy: 0.5263
Epoch 332/1000
177/177 [==============================] - 0s 80us/step - loss: 0.1504 - accuracy: 0.9661 - val_loss: 1.5880 - val_accuracy: 0.5263
Epoch 333/1000
177/177 [==============================] - 0s 83us/step - loss: 0.2408 - accuracy: 0.9548 - val_loss: 1.7152 - val_accuracy: 0.5000
Epoch 334/1000
177/177 [==============================] - 0s 99us/step - loss: 0.2043 - accuracy: 0.9718 - val_loss: 1.5551 - val_accuracy: 0.5526
Epoch 335/1000
177/177 [==============================] - 0s 84us/step - loss: 0.2463 - accuracy: 0.9548 - val_loss: 1.6085 - val_accuracy: 0.5132
Epoch 336/1000
177/177 [==============================] - 0s 88us/step - loss: 0.1725 - accuracy: 0.9605 - val_loss: 1.5931 - val_accuracy: 0.5263
Epoch 337/1000
177/177 [==============================] - 0s 90us/step - loss: 0.2603 - accuracy: 0.9492 - val_loss: 1.5958 - val_acc

177/177 [==============================] - 0s 97us/step - loss: 0.0931 - accuracy: 0.9831 - val_loss: 1.7106 - val_accuracy: 0.4868
Epoch 387/1000
177/177 [==============================] - 0s 99us/step - loss: 0.0891 - accuracy: 0.9718 - val_loss: 1.6976 - val_accuracy: 0.4868
Epoch 388/1000
177/177 [==============================] - 0s 98us/step - loss: 0.1351 - accuracy: 0.9718 - val_loss: 1.7242 - val_accuracy: 0.5000
Epoch 389/1000
177/177 [==============================] - 0s 141us/step - loss: 0.1083 - accuracy: 0.9718 - val_loss: 1.7052 - val_accuracy: 0.5263
Epoch 390/1000
177/177 [==============================] - 0s 105us/step - loss: 0.1164 - accuracy: 0.9774 - val_loss: 1.7290 - val_accuracy: 0.5263
Epoch 391/1000
177/177 [==============================] - 0s 99us/step - loss: 0.0982 - accuracy: 0.9718 - val_loss: 1.6986 - val_accuracy: 0.4868
Epoch 392/1000
177/177 [==============================] - 0s 108us/step - loss: 0.1648 - accuracy: 0.9661 - val_loss: 1.7067 - val_

177/177 [==============================] - 0s 107us/step - loss: 0.1650 - accuracy: 0.9718 - val_loss: 1.9406 - val_accuracy: 0.5132
Epoch 497/1000
177/177 [==============================] - 0s 98us/step - loss: 0.1316 - accuracy: 0.9774 - val_loss: 1.9702 - val_accuracy: 0.4868
Epoch 498/1000
177/177 [==============================] - 0s 79us/step - loss: 0.0941 - accuracy: 0.9774 - val_loss: 1.9586 - val_accuracy: 0.5395
Epoch 499/1000
177/177 [==============================] - 0s 80us/step - loss: 0.2830 - accuracy: 0.9435 - val_loss: 2.0399 - val_accuracy: 0.4474
Epoch 500/1000
177/177 [==============================] - 0s 77us/step - loss: 0.1382 - accuracy: 0.9718 - val_loss: 1.9334 - val_accuracy: 0.5132
Epoch 501/1000
177/177 [==============================] - 0s 77us/step - loss: 0.4043 - accuracy: 0.9548 - val_loss: 1.9177 - val_accuracy: 0.5132
Epoch 502/1000
177/177 [==============================] - 0s 92us/step - loss: 0.3721 - accuracy: 0.9492 - val_loss: 2.3813 - val_ac

177/177 [==============================] - 0s 107us/step - loss: 0.0765 - accuracy: 0.9774 - val_loss: 2.2523 - val_accuracy: 0.4868
Epoch 607/1000
177/177 [==============================] - 0s 115us/step - loss: 0.0938 - accuracy: 0.9774 - val_loss: 2.2065 - val_accuracy: 0.4605
Epoch 608/1000
177/177 [==============================] - 0s 118us/step - loss: 0.0632 - accuracy: 0.9831 - val_loss: 2.2616 - val_accuracy: 0.4868
Epoch 609/1000
177/177 [==============================] - 0s 129us/step - loss: 0.0796 - accuracy: 0.9831 - val_loss: 2.2112 - val_accuracy: 0.5000
Epoch 610/1000
177/177 [==============================] - 0s 106us/step - loss: 0.1185 - accuracy: 0.9718 - val_loss: 2.2123 - val_accuracy: 0.4737
Epoch 611/1000
177/177 [==============================] - 0s 109us/step - loss: 0.0841 - accuracy: 0.9831 - val_loss: 2.2527 - val_accuracy: 0.4605
Epoch 612/1000
177/177 [==============================] - 0s 133us/step - loss: 0.0749 - accuracy: 0.9774 - val_loss: 2.2259 - 

Epoch 717/1000
177/177 [==============================] - 0s 94us/step - loss: 0.1179 - accuracy: 0.9831 - val_loss: 2.4659 - val_accuracy: 0.4605
Epoch 718/1000
177/177 [==============================] - 0s 106us/step - loss: 0.2229 - accuracy: 0.9661 - val_loss: 2.3983 - val_accuracy: 0.4868
Epoch 719/1000
177/177 [==============================] - 0s 104us/step - loss: 0.1762 - accuracy: 0.9605 - val_loss: 2.5798 - val_accuracy: 0.4737
Epoch 720/1000
177/177 [==============================] - 0s 97us/step - loss: 0.2222 - accuracy: 0.9718 - val_loss: 2.4627 - val_accuracy: 0.4605
Epoch 721/1000
177/177 [==============================] - 0s 102us/step - loss: 0.1184 - accuracy: 0.9774 - val_loss: 2.4496 - val_accuracy: 0.5132
Epoch 722/1000
177/177 [==============================] - 0s 113us/step - loss: 0.1196 - accuracy: 0.9774 - val_loss: 2.5457 - val_accuracy: 0.4605
Epoch 723/1000
177/177 [==============================] - 0s 102us/step - loss: 0.1306 - accuracy: 0.9774 - val_lo

177/177 [==============================] - 0s 110us/step - loss: 0.0763 - accuracy: 0.9774 - val_loss: 2.6467 - val_accuracy: 0.4737
Epoch 828/1000
177/177 [==============================] - 0s 112us/step - loss: 0.1181 - accuracy: 0.9718 - val_loss: 2.6737 - val_accuracy: 0.5132
Epoch 829/1000
177/177 [==============================] - 0s 116us/step - loss: 0.1142 - accuracy: 0.9774 - val_loss: 2.5844 - val_accuracy: 0.4868
Epoch 830/1000
177/177 [==============================] - 0s 110us/step - loss: 0.0824 - accuracy: 0.9774 - val_loss: 2.6378 - val_accuracy: 0.4474
Epoch 831/1000
177/177 [==============================] - 0s 97us/step - loss: 0.1308 - accuracy: 0.9774 - val_loss: 2.6500 - val_accuracy: 0.4868
Epoch 832/1000
177/177 [==============================] - 0s 104us/step - loss: 0.0502 - accuracy: 0.9831 - val_loss: 2.5781 - val_accuracy: 0.4737
Epoch 833/1000
177/177 [==============================] - 0s 118us/step - loss: 0.1361 - accuracy: 0.9774 - val_loss: 2.6564 - v

177/177 [==============================] - 0s 100us/step - loss: 0.0996 - accuracy: 0.9718 - val_loss: 2.9069 - val_accuracy: 0.4605
Epoch 938/1000
177/177 [==============================] - 0s 100us/step - loss: 0.1865 - accuracy: 0.9718 - val_loss: 2.8834 - val_accuracy: 0.4474
Epoch 939/1000
177/177 [==============================] - 0s 98us/step - loss: 0.0712 - accuracy: 0.9774 - val_loss: 2.8178 - val_accuracy: 0.4868
Epoch 940/1000
177/177 [==============================] - 0s 98us/step - loss: 0.1889 - accuracy: 0.9718 - val_loss: 2.8454 - val_accuracy: 0.4342
Epoch 941/1000
177/177 [==============================] - 0s 110us/step - loss: 0.0587 - accuracy: 0.9774 - val_loss: 3.0989 - val_accuracy: 0.4474
Epoch 942/1000
177/177 [==============================] - 0s 113us/step - loss: 0.3579 - accuracy: 0.9492 - val_loss: 2.9297 - val_accuracy: 0.4474
Epoch 943/1000
177/177 [==============================] - 0s 117us/step - loss: 0.0408 - accuracy: 0.9774 - val_loss: 2.8460 - va

In [270]:
acc_test_sel4 = model_sel4.evaluate(X_sel_test, y_sel_test)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_sel4*100))

76/76 [==============================] - 0s 139us/step
over-sampling test accuracy: 53.95%


In [123]:
pred8 = model_sel4.predict_classes(X_sel_test)
pred8

array([1, 0, 2, 1, 0, 2, 1, 0, 1, 0, 2, 1, 1, 0, 2, 2, 1, 1, 2, 0, 2, 0,
       2, 2, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 2, 0,
       0, 0, 2, 1, 0, 2, 1, 1, 1, 0, 0, 2, 0, 0, 1, 2, 2, 1, 0, 2, 0, 2,
       1, 2, 2, 1, 0, 2, 1, 1, 2, 0])

In [124]:
dat8['pred'] = pred8
dat8

,strain,test,pred
130,NRS103,2,1
92,EUH25,0,0
143,NRS148,2,2
107,NRS001,1,1
166,NRS192,0,0
...,...,...,...
139,NRS113,1,2
24,BCH-SA-09,2,1
133,NRS106,2,1
62,CFBREBSa131,2,2


In [125]:
proba8 = model_sel4.predict_proba(X_sel_test)
dat_proba8 = pd.DataFrame(proba8)

In [126]:
dat_proba8

,0,1,2
0,2.431225e-07,9.923463e-01,7.653485e-03
1,1.000000e+00,7.584864e-09,1.752051e-11
2,7.180629e-07,2.044864e-03,9.979544e-01
3,3.535082e-04,9.996251e-01,2.135907e-05
4,9.998932e-01,5.361687e-05,5.318268e-05
...,...,...,...
71,2.686909e-07,9.312122e-02,9.068785e-01
72,5.529492e-07,9.999899e-01,9.559324e-06
73,1.115902e-03,9.988155e-01,6.852559e-05
74,2.708040e-07,1.567682e-01,8.432316e-01


In [127]:
dat_proba8.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba8.csv", index = False,
         header=None)

In [128]:
dat8.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/8p40pST.csv", index = False,
         header=None)

In [274]:
hist_sel4 = model_sel4.fit(X_sel_train, y_sel_train,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/1000
177/177 [==============================] - 0s 99us/step - loss: 0.0315 - accuracy: 0.9887 - val_loss: 2.7908 - val_accuracy: 0.5395
Epoch 2/1000
177/177 [==============================] - 0s 106us/step - loss: 0.0321 - accuracy: 0.9887 - val_loss: 2.7835 - val_accuracy: 0.5395
Epoch 3/1000
177/177 [==============================] - 0s 124us/step - loss: 0.0344 - accuracy: 0.9831 - val_loss: 2.7820 - val_accuracy: 0.5526
Epoch 4/1000
177/177 [==============================] - 0s 142us/step - loss: 0.0326 - accuracy: 0.9887 - val_loss: 2.7963 - val_accuracy: 0.5526
Epoch 5/1000
177/177 [==============================] - 0s 152us/step - loss: 0.0312 - accuracy: 0.9887 - val_loss: 2.8002 - val_accuracy: 0.5395
Epoch 6/1000
177/177 [==============================] - 0s 131us/step - loss: 0.0303 - accuracy: 0.9887 - val_loss: 2.7866 - val_accuracy: 0.5395
Epoch 7/1000
177/177 [==============================] - 0s 127us/step - loss: 0.

177/177 [==============================] - 0s 97us/step - loss: 0.0800 - accuracy: 0.9718 - val_loss: 3.0188 - val_accuracy: 0.5132
Epoch 112/1000
177/177 [==============================] - 0s 78us/step - loss: 0.1799 - accuracy: 0.9661 - val_loss: 3.0012 - val_accuracy: 0.5263
Epoch 113/1000
177/177 [==============================] - 0s 75us/step - loss: 0.1043 - accuracy: 0.9774 - val_loss: 2.8950 - val_accuracy: 0.5526
Epoch 114/1000
177/177 [==============================] - 0s 84us/step - loss: 0.2027 - accuracy: 0.9605 - val_loss: 2.9772 - val_accuracy: 0.5132
Epoch 115/1000
177/177 [==============================] - 0s 89us/step - loss: 0.1265 - accuracy: 0.9661 - val_loss: 2.9960 - val_accuracy: 0.5263
Epoch 116/1000
177/177 [==============================] - 0s 82us/step - loss: 0.0613 - accuracy: 0.9774 - val_loss: 2.9307 - val_accuracy: 0.5526
Epoch 117/1000
177/177 [==============================] - 0s 96us/step - loss: 0.0724 - accuracy: 0.9887 - val_loss: 2.9680 - val_acc

177/177 [==============================] - 0s 108us/step - loss: 0.0295 - accuracy: 0.9887 - val_loss: 3.0927 - val_accuracy: 0.5395
Epoch 222/1000
177/177 [==============================] - 0s 136us/step - loss: 0.0293 - accuracy: 0.9887 - val_loss: 3.1107 - val_accuracy: 0.5395
Epoch 223/1000
177/177 [==============================] - 0s 179us/step - loss: 0.0304 - accuracy: 0.9887 - val_loss: 3.0911 - val_accuracy: 0.5395
Epoch 224/1000
177/177 [==============================] - 0s 280us/step - loss: 0.0282 - accuracy: 0.9831 - val_loss: 3.0850 - val_accuracy: 0.5395
Epoch 225/1000
177/177 [==============================] - 0s 168us/step - loss: 0.0294 - accuracy: 0.9831 - val_loss: 3.0860 - val_accuracy: 0.5395
Epoch 226/1000
177/177 [==============================] - 0s 134us/step - loss: 0.0274 - accuracy: 0.9944 - val_loss: 3.0941 - val_accuracy: 0.5395
Epoch 227/1000
177/177 [==============================] - 0s 126us/step - loss: 0.0317 - accuracy: 0.9887 - val_loss: 3.0945 - 

177/177 [==============================] - 0s 244us/step - loss: 0.0291 - accuracy: 0.9831 - val_loss: 3.2546 - val_accuracy: 0.5263
Epoch 332/1000
177/177 [==============================] - 0s 250us/step - loss: 0.0263 - accuracy: 0.9887 - val_loss: 3.2676 - val_accuracy: 0.5263
Epoch 333/1000
177/177 [==============================] - 0s 115us/step - loss: 0.0279 - accuracy: 0.9831 - val_loss: 3.2630 - val_accuracy: 0.5395
Epoch 334/1000
177/177 [==============================] - 0s 117us/step - loss: 0.0269 - accuracy: 0.9887 - val_loss: 3.2602 - val_accuracy: 0.5263
Epoch 335/1000
177/177 [==============================] - 0s 109us/step - loss: 0.0270 - accuracy: 0.9887 - val_loss: 3.2577 - val_accuracy: 0.5263
Epoch 336/1000
177/177 [==============================] - 0s 119us/step - loss: 0.0280 - accuracy: 0.9887 - val_loss: 3.2625 - val_accuracy: 0.5263
Epoch 337/1000
177/177 [==============================] - 0s 118us/step - loss: 0.0269 - accuracy: 0.9887 - val_loss: 3.2606 - 

Epoch 441/1000
177/177 [==============================] - 0s 113us/step - loss: 0.0288 - accuracy: 0.9887 - val_loss: 3.4352 - val_accuracy: 0.5395
Epoch 442/1000
177/177 [==============================] - 0s 108us/step - loss: 0.0291 - accuracy: 0.9887 - val_loss: 3.4453 - val_accuracy: 0.5263
Epoch 443/1000
177/177 [==============================] - 0s 102us/step - loss: 0.0294 - accuracy: 0.9887 - val_loss: 3.4451 - val_accuracy: 0.5263
Epoch 444/1000
177/177 [==============================] - 0s 219us/step - loss: 0.0263 - accuracy: 0.9887 - val_loss: 3.4187 - val_accuracy: 0.5263
Epoch 445/1000
177/177 [==============================] - 0s 356us/step - loss: 0.0270 - accuracy: 0.9887 - val_loss: 3.4159 - val_accuracy: 0.5263
Epoch 446/1000
177/177 [==============================] - 0s 209us/step - loss: 0.0297 - accuracy: 0.9831 - val_loss: 3.4297 - val_accuracy: 0.5132
Epoch 447/1000
177/177 [==============================] - 0s 316us/step - loss: 0.0277 - accuracy: 0.9831 - val_

177/177 [==============================] - 0s 157us/step - loss: 0.2640 - accuracy: 0.9718 - val_loss: 3.5392 - val_accuracy: 0.5263
Epoch 552/1000
177/177 [==============================] - 0s 134us/step - loss: 0.0720 - accuracy: 0.9774 - val_loss: 3.4923 - val_accuracy: 0.5395
Epoch 553/1000
177/177 [==============================] - 0s 247us/step - loss: 0.1415 - accuracy: 0.9774 - val_loss: 3.4437 - val_accuracy: 0.5526
Epoch 554/1000
177/177 [==============================] - 0s 391us/step - loss: 0.0355 - accuracy: 0.9831 - val_loss: 3.4980 - val_accuracy: 0.5395
Epoch 555/1000
177/177 [==============================] - 0s 127us/step - loss: 0.0577 - accuracy: 0.9831 - val_loss: 3.5169 - val_accuracy: 0.5395
Epoch 556/1000
177/177 [==============================] - 0s 127us/step - loss: 0.2164 - accuracy: 0.9774 - val_loss: 3.5363 - val_accuracy: 0.5395
Epoch 557/1000
177/177 [==============================] - 0s 222us/step - loss: 0.2389 - accuracy: 0.9548 - val_loss: 3.5145 - 

177/177 [==============================] - 0s 128us/step - loss: 0.0291 - accuracy: 0.9887 - val_loss: 3.6429 - val_accuracy: 0.5395
Epoch 662/1000
177/177 [==============================] - 0s 110us/step - loss: 0.0278 - accuracy: 0.9887 - val_loss: 3.6466 - val_accuracy: 0.5263
Epoch 663/1000
177/177 [==============================] - 0s 114us/step - loss: 0.0264 - accuracy: 0.9831 - val_loss: 3.6505 - val_accuracy: 0.5263
Epoch 664/1000
177/177 [==============================] - 0s 115us/step - loss: 0.0269 - accuracy: 0.9831 - val_loss: 3.6490 - val_accuracy: 0.5263
Epoch 665/1000
177/177 [==============================] - 0s 127us/step - loss: 0.0246 - accuracy: 0.9887 - val_loss: 3.6484 - val_accuracy: 0.5263
Epoch 666/1000
177/177 [==============================] - 0s 114us/step - loss: 0.0271 - accuracy: 0.9887 - val_loss: 3.6555 - val_accuracy: 0.5263
Epoch 667/1000
177/177 [==============================] - 0s 129us/step - loss: 0.0257 - accuracy: 0.9887 - val_loss: 3.6527 - 

177/177 [==============================] - 0s 116us/step - loss: 0.0254 - accuracy: 0.9887 - val_loss: 3.8066 - val_accuracy: 0.5263
Epoch 772/1000
177/177 [==============================] - 0s 118us/step - loss: 0.0255 - accuracy: 0.9887 - val_loss: 3.8078 - val_accuracy: 0.5263
Epoch 773/1000
177/177 [==============================] - 0s 112us/step - loss: 0.0248 - accuracy: 0.9831 - val_loss: 3.8142 - val_accuracy: 0.5263
Epoch 774/1000
177/177 [==============================] - 0s 124us/step - loss: 0.0265 - accuracy: 0.9887 - val_loss: 3.8059 - val_accuracy: 0.5263
Epoch 775/1000
177/177 [==============================] - 0s 139us/step - loss: 0.0270 - accuracy: 0.9887 - val_loss: 3.8162 - val_accuracy: 0.5263
Epoch 776/1000
177/177 [==============================] - 0s 123us/step - loss: 0.0253 - accuracy: 0.9887 - val_loss: 3.8058 - val_accuracy: 0.5263
Epoch 777/1000
177/177 [==============================] - 0s 118us/step - loss: 0.0253 - accuracy: 0.9887 - val_loss: 3.8019 - 

177/177 [==============================] - 0s 220us/step - loss: 0.0287 - accuracy: 0.9831 - val_loss: 3.9587 - val_accuracy: 0.5263
Epoch 882/1000
177/177 [==============================] - 0s 132us/step - loss: 0.0265 - accuracy: 0.9887 - val_loss: 3.9494 - val_accuracy: 0.5263
Epoch 883/1000
177/177 [==============================] - 0s 195us/step - loss: 0.0255 - accuracy: 0.9887 - val_loss: 3.9303 - val_accuracy: 0.5263
Epoch 884/1000
177/177 [==============================] - 0s 148us/step - loss: 0.0242 - accuracy: 0.9887 - val_loss: 3.9283 - val_accuracy: 0.5263
Epoch 885/1000
177/177 [==============================] - 0s 132us/step - loss: 0.0263 - accuracy: 0.9887 - val_loss: 3.9401 - val_accuracy: 0.5263
Epoch 886/1000
177/177 [==============================] - 0s 120us/step - loss: 0.0246 - accuracy: 0.9887 - val_loss: 3.9308 - val_accuracy: 0.5263
Epoch 887/1000
177/177 [==============================] - 0s 116us/step - loss: 0.0255 - accuracy: 0.9887 - val_loss: 3.9383 - 

177/177 [==============================] - 0s 193us/step - loss: 0.0268 - accuracy: 0.9831 - val_loss: 4.1355 - val_accuracy: 0.5132
Epoch 992/1000
177/177 [==============================] - 0s 156us/step - loss: 0.0255 - accuracy: 0.9887 - val_loss: 4.1392 - val_accuracy: 0.5132
Epoch 993/1000
177/177 [==============================] - 0s 154us/step - loss: 0.0261 - accuracy: 0.9887 - val_loss: 4.1475 - val_accuracy: 0.5132
Epoch 994/1000
177/177 [==============================] - 0s 119us/step - loss: 0.0261 - accuracy: 0.9831 - val_loss: 4.1588 - val_accuracy: 0.5132
Epoch 995/1000
177/177 [==============================] - 0s 150us/step - loss: 0.0289 - accuracy: 0.9887 - val_loss: 4.1549 - val_accuracy: 0.5263
Epoch 996/1000
177/177 [==============================] - 0s 199us/step - loss: 0.0281 - accuracy: 0.9887 - val_loss: 4.1430 - val_accuracy: 0.5132
Epoch 997/1000
177/177 [==============================] - 0s 139us/step - loss: 0.0254 - accuracy: 0.9887 - val_loss: 4.1259 - 

In [275]:
print('train accuracy: %.2f%%' % (np.mean(hist_sel4.history['accuracy'])*100))

train accuracy: 98.35%


In [104]:
df_proba8 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_lasso_2.xlsx",
                        sheet_name=3,
                        index_col=None)

In [105]:
df_proba8

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,NRS104,0,1,8.210638e-02,0.917809,8.477923e-05
1,p0017kpresabs_qual,NRS071,0,2,1.015229e-02,0.255327,7.345203e-01
2,p0017kpresabs_qual,NRS072,1,2,3.642946e-03,0.000002,9.963547e-01
3,p0017kpresabs_qual,BCH-SA-12,0,2,2.809318e-03,0.099188,8.980029e-01
4,p0017kpresabs_qual,CFBREBSa125,2,0,4.875667e-01,0.088657,4.237761e-01
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,NRS113,1,2,2.686909e-07,0.093121,9.068785e-01
604,p0040presabsSTCC_qual,BCH-SA-09,2,1,5.529492e-07,0.999990,9.559324e-06
605,p0040presabsSTCC_qual,NRS106,2,1,1.115902e-03,0.998816,6.852559e-05
606,p0040presabsSTCC_qual,CFBREBSa131,2,2,2.708040e-07,0.156768,8.432316e-01


In [106]:
y_prob8 = df_proba8[df_proba8['phage']=='p0040presabsSTCC_qual'].iloc[:,-3:]
y_prob8 = y_prob8.to_numpy()
y_prob8

array([[2.43122490e-07, 9.92346300e-01, 7.65348460e-03],
       [1.00000000e+00, 7.58486400e-09, 1.75205150e-11],
       [7.18062900e-07, 2.04486400e-03, 9.97954370e-01],
       [3.53508200e-04, 9.99625100e-01, 2.13590670e-05],
       [9.99893200e-01, 5.36168740e-05, 5.31826770e-05],
       [1.06981310e-02, 2.92876200e-01, 6.96425600e-01],
       [2.58759500e-06, 9.54899200e-01, 4.50981900e-02],
       [9.07146800e-01, 2.17483380e-02, 7.11049140e-02],
       [1.27781110e-05, 6.22910260e-01, 3.77076950e-01],
       [8.01830950e-01, 1.69265790e-01, 2.89032240e-02],
       [4.29922060e-03, 9.91900600e-03, 9.85781850e-01],
       [2.15574220e-07, 9.99999760e-01, 7.08923960e-11],
       [2.29871820e-05, 9.99974850e-01, 2.18163500e-06],
       [9.99760700e-01, 9.99083400e-06, 2.29271700e-04],
       [6.13850000e-04, 3.45332970e-03, 9.95932900e-01],
       [3.76958470e-03, 8.33303100e-03, 9.87897400e-01],
       [1.58768310e-04, 6.10627230e-01, 3.89213950e-01],
       [1.28875790e-02, 9.86618

In [107]:
ovo8 = rocauc_ovo(y_sel_test, y_prob8, average="macro", multi_class="ovo")
ovo8

0.6741404257452303

In [108]:
ovr8 = rocauc_ovr(y_sel_test, y_prob8, average="macro", multi_class="ovr")
ovr8

0.6741404257452303

In [109]:
ovos2 = [ovo5, ovo6, ovo7, ovo8]
np.mean(ovos2)

0.6892898964911329

In [110]:
np.std(ovos2)

0.010890681093656675

In [111]:
ovrs2 = [ovr5, ovr6, ovr7, ovr8]
np.mean(ovrs2)

0.6892898964911329

In [112]:
np.std(ovrs2)

0.010890681093656675

In [276]:
accs_l= [acc_test_sel, acc_test_sel2, acc_test_sel3, acc_test_sel4]

In [277]:
mean_l = np.mean(accs_l)
print('test accuracy mean after lasso: %.2f%%' % (mean_l*100))

test accuracy mean after lasso: 54.61%


In [278]:
std_l = np.std(accs_l)
print('test accuracy standard deviation after lasso:', std_l)

test accuracy standard deviation after lasso: 0.019736831386885305


In [279]:
accs_train_l = [np.mean(hist_sel.history['accuracy']), np.mean(hist_sel2.history['accuracy']), np.mean(hist_sel3.history['accuracy']),
             np.mean(hist_sel4.history['accuracy'])]

In [280]:
mean_train_l = np.mean(accs_train_l)
print('train accuracy mean after lasso: %.2f%%' % (mean_train_l*100))

train accuracy mean after lasso: 98.36%


In [281]:
std_train_l = np.std(accs_train_l)
print('train accuracy standard deviation after lasso:', std_train_l)

train accuracy standard deviation after lasso: 0.009281495
